# 1 导入必要库和设置环境

In [1]:
import json
import os
import random
import math  # 确保balance_classify_data函数使用math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from torch.optim import AdamW
from collections import Counter  # 添加这行导入
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from IPython.display import display, HTML
import sys

# 添加项目根目录到Python路径
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

# 导入我们自己的Seq2SeqTransformer模型
from src.models.core_seq2seq import Seq2SeqTransformer, PositionalEncoding

# 设置随机种子以保证可重复性
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# 检测设备
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"使用GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU内存总量: {torch.cuda.get_device_properties(0).total_memory / 1024 / 1024 / 1024:.2f} GB")
        print(f"可用GPU数量: {torch.cuda.device_count()}")
    else:
        device = torch.device("cpu")
        print("使用CPU")
    
    return device

/root/miniconda3/envs/multi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess_mrc_data(data):
    """预处理MRC数据，增加[SEP]分隔符，确保问题和上下文分离"""
    for item in data:
        if item['type'] == 'mrc':
            # 检查是否已有[SEP]
            if '[SEP]' not in item['input']:
                # 尝试从文本中提取问题部分
                text = item['input']
                target = item['target']
                
                # 常见问句前缀
                question_markers = ["什么是", "谁是", "哪个", "怎样", "如何", "为什么", 
                                   "什么时候", "在哪里", "请问"]
                
                # 尝试方法1: 查找问句标记
                question = None
                for marker in question_markers:
                    marker_pos = text.find(marker)
                    if marker_pos != -1:
                        # 寻找完整问句
                        end_pos = text.find("？", marker_pos)
                        if end_pos != -1:
                            # 提取问题，保留问号
                            question = text[marker_pos:end_pos+1]
                            # 删除问题，留下上下文
                            context = text[:marker_pos] + text[end_pos+1:]
                            item['input'] = context + " [SEP] " + question
                            break
                
                # 方法2: 使用目标答案定位问题
                if question is None and target in text:
                    target_pos = text.find(target)
                    # 查看答案之后是否有问号
                    question_end = text.find("？", target_pos)
                    if question_end != -1:
                        # 向前查找可能的问题开始
                        question_start = max(0, question_end - 20)
                        question = text[question_start:question_end+1]
                        # 移除问题部分，留下上下文
                        context = text[:question_start] + text[question_end+1:]
                        item['input'] = context + " [SEP] " + question
                
                # 方法3: 如果无法检测到问题，创建一个基于答案的合成问题
                if question is None:
                    if target in text:
                        # 生成一个关于答案的通用问题
                        question = f"什么是{target}？" if len(target) < 10 else "文中的重要信息是什么？"
                        item['input'] = text + " [SEP] " + question
                    else:
                        # 答案不在文本中，创建一个通用问题
                        question = "文中的重要信息是什么？"
                        # 将答案插入文本中
                        mid_point = len(text) // 2
                        text = text[:mid_point] + target + text[mid_point:]
                        item['input'] = text + " [SEP] " + question
    
    return data

In [3]:
def mrc_data_augmentation(dataset, augment_ratio=0.3):
    """增强MRC任务数据"""
    augmented_data = []
    mrc_items = [item for item in dataset if item['type'] == 'mrc']
    
    # 确定需要增强的样本数量
    aug_samples = int(len(mrc_items) * augment_ratio)
    
    # 随机选择用于增强的样本
    selected_samples = random.sample(mrc_items, min(aug_samples, len(mrc_items)))
    
    for item in selected_samples:
        # 策略1: 交换答案周围的词语顺序
        aug_item1 = _swap_answer_context(item)
        if aug_item1:
            augmented_data.append(aug_item1)
            
        # 策略2: 使用同义词替换
        aug_item2 = _synonym_replacement(item)
        if aug_item2:
            augmented_data.append(aug_item2)
        
        # 策略3: 自问自答增强
        aug_item3 = _self_qa_augmentation(item)
        if aug_item3:
            augmented_data.append(aug_item3)
    
    # 将增强的数据添加到原始数据集
    return dataset + augmented_data

def _swap_answer_context(item):
    """交换答案周围的词语顺序"""
    input_text = item['input']
    target = item['target']
    
    # 只在找得到答案时进行增强
    if target not in input_text:
        return None
        
    target_pos = input_text.find(target)
    context_before = input_text[:target_pos].strip()
    context_after = input_text[target_pos+len(target):].strip()
    
    # 如果有足够的上下文用于交换
    if len(context_before) > 10 and len(context_after) > 10:
        # 将答案前后的上下文交换位置
        new_input = context_after + " " + target + " " + context_before
        
        # 构造新问题
        question = f"什么是{target}？" if len(target) < 10 else "文中的重要信息是什么？"
        
        # 添加SEP分隔符
        new_input = new_input + " [SEP] " + question
        
        return {
            'type': 'mrc',
            'input': new_input,
            'target': target,
            'answer_choices': item.get('answer_choices', [])
        }
    
    return None

def _synonym_replacement(item):
    """使用同义词替换文本中的一些词语"""
    # 简单的中文同义词词典
    synonyms = {
        "大": ["巨大", "庞大", "宏伟"],
        "小": ["微小", "细小", "迷你"],
        "好": ["良好", "优秀", "不错"],
        "坏": ["糟糕", "恶劣", "不好"],
        "说": ["讲", "表示", "告诉"],
        "快": ["迅速", "敏捷", "迅捷"],
        "慢": ["缓慢", "迟缓", "迟钝"],
        "美丽": ["漂亮", "好看", "悦目"],
        "重要": ["关键", "主要", "核心"],
        # 可以添加更多同义词
    }
    
    input_text = item['input']
    target = item['target']
    
    # 创建一个新的文本副本
    new_text = input_text
    
    # 确保不替换答案部分
    if target in new_text:
        target_pos = new_text.find(target)
        pre_text = new_text[:target_pos]
        post_text = new_text[target_pos + len(target):]
        
        # 在答案前后的文本中进行词语替换
        for word, syn_list in synonyms.items():
            if word in pre_text:
                pre_text = pre_text.replace(word, random.choice(syn_list), 1)
            
            if word in post_text:
                post_text = post_text.replace(word, random.choice(syn_list), 1)
        
        new_text = pre_text + target + post_text
    else:
        # 如果找不到答案，随机替换几个词
        for word, syn_list in synonyms.items():
            if word in new_text:
                new_text = new_text.replace(word, random.choice(syn_list), 1)
    
    if new_text != input_text:
        # 确保有SEP分隔符
        if '[SEP]' not in new_text:
            question = f"什么是{target}？" if len(target) < 10 else "文中的重要信息是什么？"
            new_text = new_text + " [SEP] " + question
            
        return {
            'type': 'mrc',
            'input': new_text,
            'target': target,
            'answer_choices': item.get('answer_choices', [])
        }
    
    return None

def _self_qa_augmentation(item):
    """使用答案创建问题，构建更明确的QA格式"""
    input_text = item['input']
    target = item['target']
    
    # 只处理找得到答案的情况
    if target not in input_text:
        return None
    
    # 根据答案特点生成问题
    if len(target) <= 4:
        question = f"什么是{target}？"
    elif '人' in target or '者' in target:
        question = f"谁是{target[:5] if len(target) > 5 else target}？"
    elif '年' in target or '月' in target or '日' in target:
        question = "这件事发生在什么时间？"
    elif '地' in target or '处' in target or '国' in target:
        question = "这件事发生在哪里？"
    else:
        question = f"请问{target[:5] if len(target) > 5 else target}是什么意思？"
    
    # 构建新的输入，确保使用SEP分隔符
    context = input_text.replace(target, "{" + target + "}")  # 标记答案位置
    new_input = context + " [SEP] " + question
    
    return {
        'type': 'mrc',
        'input': new_input,
        'target': target,
        'answer_choices': item.get('answer_choices', [])
    }

# 2 定义辅助函数

## 2.1 数据加载

In [4]:
from concurrent.futures import ProcessPoolExecutor
import math
import os

def parallel_load_data(file_path, sample_percentage=10, max_samples=None, task_types=None, n_workers=8):
    """并行加载并抽样JSON数据文件，处理编码问题"""
    print(f"Reading file and sampling {sample_percentage}% of data using {n_workers} workers...")
    
    # 获取文件大小
    file_size = os.path.getsize(file_path)
    
    # 分片大小计算
    chunk_size = math.ceil(file_size / n_workers)
    
    # 创建任务列表
    tasks = []
    with open(file_path, 'rb') as f:  # 使用二进制模式打开，避免编码问题
        start_pos = 0
        while start_pos < file_size:
            f.seek(start_pos)
            if start_pos > 0:
                # 读取到下一个换行符，确保不会截断一行
                line = b''
                char = f.read(1)
                while char != b'\n' and char != b'':
                    line += char
                    char = f.read(1)
            
            current_pos = f.tell()
            tasks.append((file_path, current_pos, min(current_pos + chunk_size, file_size),
                         sample_percentage, max_samples // n_workers if max_samples else None, task_types))
            start_pos = current_pos + chunk_size
    
    # 使用进程池并行处理
    sampled_data = []
    total_lines = 0
    with ProcessPoolExecutor(max_workers=n_workers) as executor:
        for result in executor.map(process_file_chunk, tasks):
            chunk_data, chunk_lines = result
            sampled_data.extend(chunk_data)
            total_lines += chunk_lines
    
    # 如果有最大样本限制，对结果进行截断
    if max_samples and len(sampled_data) > max_samples:
        sampled_data = sampled_data[:max_samples]
    
    print(f"Total lines read: {total_lines}")
    print(f"Sampled data size: {len(sampled_data)}")
    
    return sampled_data

def process_file_chunk(args):
    """处理文件的一个分块，使用二进制模式避免编码错误"""
    file_path, start, end, sample_percentage, max_samples, task_types = args
    sample_prob = sample_percentage / 100
    sampled_data = []
    line_count = 0
    
    # 以二进制模式打开文件
    with open(file_path, 'rb') as f:
        f.seek(start)
        
        # 读取直到结束位置
        while f.tell() < end:
            line_bytes = f.readline()
            if not line_bytes:  # 文件结束
                break
            
            line_count += 1
            
            # 尝试解码行
            try:
                line = line_bytes.decode('utf-8').strip()
                
                # 随机抽样
                if random.random() <= sample_prob:
                    try:
                        item = json.loads(line)
                        
                        if task_types is not None and item.get('type') not in task_types:
                            continue
                        
                        sampled_data.append(item)
                        
                        # 检查是否达到最大样本数
                        if max_samples and len(sampled_data) >= max_samples:
                            break
                    
                    except json.JSONDecodeError:
                        continue
            except UnicodeDecodeError:
                # 忽略无法解码的行
                continue
    
    return sampled_data, line_count

In [5]:
# 流式加载JSON数据并随机抽样
def load_json_data_with_sampling(file_path, sample_percentage=10, max_samples=None, task_types=None):
    sample_prob = sample_percentage / 100
    sampled_data = []
    line_count = 0
    
    print(f"Reading file and sampling {sample_percentage}% of data...")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f):
            line_count += 1
            
            if random.random() <= sample_prob:
                try:
                    item = json.loads(line.strip())
                    
                    if task_types is not None and item.get('type') not in task_types:
                        continue
                        
                    sampled_data.append(item)
                    
                    if max_samples and len(sampled_data) >= max_samples:
                        break
                except json.JSONDecodeError:
                    continue

    print(f"Total lines read: {line_count}")
    print(f"Sampled data size: {len(sampled_data)}")
    
    return sampled_data

## 2.2 标签映射检查

In [6]:
# 检查标签配置
def fix_label_mapping(data, task_types):
    task_specific_labels = {t: set() for t in task_types}
    
    # 统计每种任务的标签
    for item in data:
        task = item['type']
        if task in task_types and 'answer_choices' in item:
            for choice in item['answer_choices']:
                task_specific_labels[task].add(choice)
            
    print("任务特定标签统计:")
    for task, labels in task_specific_labels.items():
        print(f"{task}: {len(labels)} 个标签 - {list(labels)[:5]}...")

## 2.3 模型评估

In [7]:
# 评估函数 - 修改以支持MRC任务
def evaluate(model, dataloader, device, tokenizer):
    model.eval()
    
    all_preds = []
    all_labels = []
    all_task_types = []
    mrc_results = []  # 存储MRC任务的结果
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch.get('attention_mask', None)
            if attention_mask is not None:
                attention_mask = attention_mask.to(device)
            src_padding_mask = batch.get('src_padding_mask', None)
            if src_padding_mask is not None:
                src_padding_mask = src_padding_mask.to(device)
            labels = batch['label'].to(device)
            task_types = batch['task_type']
            target_texts = batch['target_text']
            
            valid_indices = (labels != -100).nonzero(as_tuple=True)[0]
            mrc_indices = [i for i, t in enumerate(task_types) if t == 'mrc']
            
            # 处理分类和NLI任务
            if len(valid_indices) > 0:
                batch_input_ids = input_ids[valid_indices]
                batch_attention_mask = attention_mask[valid_indices] if attention_mask is not None else None
                batch_src_padding_mask = src_padding_mask[valid_indices] if src_padding_mask is not None else None
                batch_labels = labels[valid_indices]
                batch_task_types = [task_types[i] for i in valid_indices]
                
                # 只处理非MRC任务
                non_mrc_indices = [i for i, t in enumerate(batch_task_types) if t != 'mrc']
                if non_mrc_indices:
                    non_mrc_input_ids = batch_input_ids[non_mrc_indices]
                    non_mrc_attention_mask = batch_attention_mask[non_mrc_indices] if batch_attention_mask is not None else None
                    non_mrc_token_type_ids = batch.get('token_type_ids', None)
                    if non_mrc_token_type_ids is not None:
                        cpu_valid_indices = valid_indices.cpu()
                        non_mrc_token_type_ids = non_mrc_token_type_ids[cpu_valid_indices][non_mrc_indices].to(device)
                    
                    non_mrc_task_types = [batch_task_types[i] for i in non_mrc_indices]
                    non_mrc_labels = batch_labels[non_mrc_indices]
                    
                    with torch.cuda.amp.autocast():
                        outputs = model(
                            input_ids=non_mrc_input_ids,
                            attention_mask=non_mrc_attention_mask,
                            token_type_ids=non_mrc_token_type_ids,
                            task_type=non_mrc_task_types
                        )
                    
                    preds = torch.argmax(outputs, dim=-1)
                    preds = preds.cpu().numpy()
                    non_mrc_labels = non_mrc_labels.cpu().numpy()
                    
                    all_preds.extend(preds)
                    all_labels.extend(non_mrc_labels)
                    all_task_types.extend(non_mrc_task_types)
            
            # 单独处理MRC任务
            if mrc_indices:
                for i in mrc_indices:
                    mrc_input_ids = input_ids[i:i+1]
                    mrc_attention_mask = attention_mask[i:i+1] if attention_mask is not None else None
                    mrc_token_type_ids = batch.get('token_type_ids', None)
                    if mrc_token_type_ids is not None:
                        mrc_token_type_ids = mrc_token_type_ids[i:i+1].to(device)
                    
                    with torch.cuda.amp.autocast():
                        start_logits, end_logits = model(
                            input_ids=mrc_input_ids,
                            attention_mask=mrc_attention_mask,
                            token_type_ids=mrc_token_type_ids,
                            task_type='mrc'
                        )
                    
                    # 获取最可能的起始和结束位置
                    start_idx = torch.argmax(start_logits, dim=-1).item()
                    end_idx = torch.argmax(end_logits, dim=-1).item()
                    
                    # 确保end_idx >= start_idx
                    if end_idx < start_idx:
                        end_idx = start_idx
                    
                    # 将token ids转换为文本
                    predicted_answer = tokenizer.decode(mrc_input_ids[0][start_idx:end_idx+1].tolist(),
                                                     skip_special_tokens=True)
                    
                    # 计算精确匹配分数 (EM)
                    target_text = target_texts[i]
                    is_exact_match = predicted_answer.strip() == target_text.strip()
                    
                    mrc_results.append({
                        'predicted': predicted_answer,
                        'target': target_text,
                        'exact_match': is_exact_match
                    })
    
    results = {}
    if all_preds:
        overall_acc = accuracy_score(all_labels, all_preds)
        results['overall_accuracy'] = overall_acc
        
        # 计算F1分数
        f1 = f1_score(all_labels, all_preds, average='weighted')
        results['overall_f1'] = f1
    
    # 计算任务特定指标
    task_metrics = {}
    for task_type in set(all_task_types):
        task_indices = [i for i, t in enumerate(all_task_types) if t == task_type]
        if task_indices:
            task_preds = [all_preds[i] for i in task_indices]
            task_labels = [all_labels[i] for i in task_indices]
            task_acc = accuracy_score(task_labels, task_preds)
            task_f1 = f1_score(task_labels, task_preds, average='weighted')
            
            results[f'{task_type}_accuracy'] = task_acc
            results[f'{task_type}_f1'] = task_f1
            
            task_metrics[task_type] = {
                'accuracy': task_acc,
                'f1': task_f1,
                'support': len(task_indices)
            }
    
    # 计算MRC任务的精确匹配分数
    if mrc_results:
        mrc_em = sum(item['exact_match'] for item in mrc_results) / len(mrc_results)
        results['mrc_em'] = mrc_em
        task_metrics['mrc'] = {
            'exact_match': mrc_em,
            'support': len(mrc_results)
        }
    
    # 在Kaggle中显示详细结果表格
    if task_metrics:
        metrics_df = pd.DataFrame.from_dict(task_metrics, orient='index')
        display(HTML("<h4>Detailed Task Metrics</h4>"))
        display(metrics_df)
    
    return results

In [8]:
def evaluate_multitask(model, val_loaders, device, tokenizer):
    """评估多任务模型"""
    # 添加异常处理
    try:
        model.eval()
        
        task_results = {}
        
        with torch.no_grad():
            for task_type, loader in val_loaders.items():
                # 每个任务类型开始前清理内存
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    
                if task_type in ['classify', 'nli']:
                    all_preds = []
                    all_labels = []
                    all_target_texts = []  # 新增：保存目标文本
                    
                    # 每处理10个批次清理一次内存
                    for batch_idx, batch in enumerate(tqdm(loader, desc=f"Evaluating {task_type}")):
                        input_ids = batch['input_ids'].to(device)
                        attention_mask = batch.get('attention_mask', None)
                        if attention_mask is not None:
                            attention_mask = attention_mask.to(device)
                        token_type_ids = batch.get('token_type_ids', None)
                        if token_type_ids is not None:
                            token_type_ids = token_type_ids.to(device)
                        labels = batch['label'].to(device)
                        
                        # 新增：保存目标文本
                        if 'target_text' in batch:
                            all_target_texts.extend(batch['target_text'])
                        
                        outputs = model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            task_type=task_type
                        )
                        
                        preds = torch.argmax(outputs, dim=-1)
                        # 每个样本单独处理，减少峰值内存使用
                        for i in range(preds.size(0)):
                            all_preds.append(preds[i].item())  # 直接取值而不是转numpy
                            all_labels.append(labels[i].item())
                        
                        if torch.cuda.is_available() and batch_idx % 10 == 0:
                            torch.cuda.empty_cache()
                    
                    if all_preds:
                        accuracy = accuracy_score(all_labels, all_preds)
                        f1 = f1_score(all_labels, all_preds, average='weighted')
                        task_results[f'{task_type}_accuracy'] = accuracy
                        task_results[f'{task_type}_f1'] = f1
                        
                        # 新增：计算分类任务的详细指标
                        if task_type == 'classify' and all_target_texts:
                            # 获取标签分布
                            label_distribution = Counter(all_target_texts)
                            
                            # 计算类别级别指标
                            class_metrics = {}
                            for label in set(all_target_texts):
                                # 找出这个类别的样本索引
                                indices = [i for i, t in enumerate(all_target_texts) if t == label]
                                if indices:
                                    # 计算这个类别的准确率
                                    class_preds = [all_preds[i] for i in indices]
                                    class_labels = [all_labels[i] for i in indices]
                                    class_acc = accuracy_score(class_labels, class_preds)
                                    class_metrics[label] = {
                                        'count': len(indices),
                                        'accuracy': class_acc
                                    }
                            
                            # 计算罕见和常见类别的平均准确率
                            if len(class_metrics) > 5:  # 至少有5个类别才统计
                                # 按数量排序，取前20%作为常见类，后20%作为罕见类
                                sorted_classes = sorted(class_metrics.items(), key=lambda x: x[1]['count'])
                                total_classes = len(sorted_classes)
                                
                                # 罕见类
                                rare_count = max(1, total_classes // 5)
                                rare_classes = sorted_classes[:rare_count]
                                
                                # 常见类
                                common_classes = sorted_classes[-rare_count:]
                                
                                # 计算准确率
                                if rare_classes:
                                    rare_acc = sum(metrics['accuracy'] for _, metrics in rare_classes) / len(rare_classes)
                                    task_results[f'{task_type}_rare_accuracy'] = rare_acc
                                    
                                if common_classes:
                                    common_acc = sum(metrics['accuracy'] for _, metrics in common_classes) / len(common_classes)
                                    task_results[f'{task_type}_common_accuracy'] = common_acc
                                    
                                # 计算平衡准确率 (各类别平均)
                                balanced_acc = sum(metrics['accuracy'] for _, metrics in class_metrics.items()) / len(class_metrics)
                                task_results[f'{task_type}_balanced_accuracy'] = balanced_acc
                        
                elif task_type == 'mrc':
                    mrc_results = []
                    
                    # 每处理10个批次清理一次内存
                    for batch_idx, batch in enumerate(tqdm(loader, desc="Evaluating MRC")):
                        input_ids = batch['input_ids'].to(device)
                        attention_mask = batch.get('attention_mask', None)
                        if attention_mask is not None:
                            attention_mask = attention_mask.to(device)
                        token_type_ids = batch.get('token_type_ids', None)
                        if token_type_ids is not None:
                            token_type_ids = token_type_ids.to(device)
                        target_texts = batch['target_text']
                        
                        start_logits, end_logits = model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            task_type='mrc'
                        )
                        
                        # 对每个样本计算预测答案
                        for i in range(input_ids.size(0)):
                            sample_start_logits = start_logits[i]
                            sample_end_logits = end_logits[i]
                            
                            # 获取最可能的起始和结束位置
                            start_idx = torch.argmax(sample_start_logits).item()
                            end_idx = torch.argmax(sample_end_logits).item()
                            
                            # 确保end_idx >= start_idx
                            if end_idx < start_idx:
                                end_idx = start_idx
                                
                            # 限制答案长度
                            if end_idx - start_idx > 30:
                                end_idx = start_idx + 30
                            
                            # 将token ids转换为文本
                            predicted_answer = tokenizer.decode(
                                input_ids[i][start_idx:end_idx+1].tolist(),
                                skip_special_tokens=True,
                                clean_up_tokenization_spaces=True
                            )
                            # 后续处理步骤
                            predicted_answer = predicted_answer.replace(" ", "")  # 移除空格
                            predicted_answer = postprocess_mrc_answer(predicted_answer, target_texts[i])  # 添加这行
                            
                            # 计算精确匹配
                            target_text = target_texts[i]
                            is_exact_match = predicted_answer.strip() == target_text.strip()
                            
                            mrc_results.append({
                                'predicted': predicted_answer,
                                'target': target_text,
                                'exact_match': is_exact_match
                            })
                        
                        if torch.cuda.is_available() and batch_idx % 10 == 0:
                            torch.cuda.empty_cache()
                    
                    # 记录更详细的结果
                    if mrc_results:
                        predicted_texts = [item['predicted'] for item in mrc_results]
                        target_texts = [item['target'] for item in mrc_results]
                        
                        # 计算多种指标
                        metrics = calculate_mrc_metrics(predicted_texts, target_texts)
                        exact_match = metrics["exact_match"]
                        f1 = metrics["f1"]
                        partial_match = metrics["partial_match"]
    
                        # 展示详细诊断信息
                        print(f"\n【MRC诊断信息】")
                        print(f"样本数: {len(mrc_results)}")
                        print(f"精确匹配率: {exact_match:.4f}")
                        print(f"平均F1分数: {f1:.4f}")
                        print(f"部分匹配率: {partial_match:.4f}")
                        
                        # 展示一些样例
                        print("\n【预测样例】")
                        for i in range(min(5, len(mrc_results))):
                            print(f"目标: '{mrc_results[i]['target']}'")
                            print(f"预测: '{mrc_results[i]['predicted']}'")
                            print(f"匹配: {mrc_results[i]['exact_match']}\n")
                        
                        task_results['mrc_em'] = exact_match
                        task_results['mrc_f1'] = f1
                        task_results['mrc_partial'] = partial_match
        
        # 计算总体结果
        if task_results:
            task_results['overall_accuracy'] = sum(
                value for key, value in task_results.items() 
                if key.endswith('_accuracy') or key.endswith('_em')
            ) / sum(1 for key in task_results if key.endswith('_accuracy') or key.endswith('_em'))
        
        # 显示详细结果表格
        metrics_df = pd.DataFrame(
            [[task_results.get(f'{task}_accuracy', None), 
              task_results.get(f'{task}_f1', None), 
              task_results.get(f'{task}_em', None)] 
             for task in ['classify', 'nli', 'mrc']],
            index=['Classify', 'NLI', 'MRC'],
            columns=['Accuracy/EM', 'F1', 'EM']
        )
        display(HTML("<h4>Detailed Task Metrics</h4>"))
        display(metrics_df)
        
        return task_results
        
    except RuntimeError as e:
        if 'out of memory' in str(e):
            print('| WARNING: 内存不足，尝试减小批次大小')
            torch.cuda.empty_cache()
            # 返回简单结果避免完全失败
            return {"overall_accuracy": 0, "classify_accuracy": 0, "nli_accuracy": 0, "mrc_em": 0}
        else:
            raise e

# 3 创建数据集和数据加载器

## 3.1 数据集类

In [9]:
def balance_classify_data_improved(classify_data, max_per_class=3000, min_per_class=300):
    """增强版分类数据平衡函数 - 更积极地处理稀有类别"""
    # 按标签分组
    label_groups = {}
    for item in classify_data:
        target = item['target']
        if target not in label_groups:
            label_groups[target] = []
        label_groups[target].append(item)
    
    # 计算类别分布
    label_counts = {label: len(items) for label, items in label_groups.items()}
    print(f"分类数据共有 {len(label_counts)} 个不同的类别")
    
    # 分析类别分布
    count_distribution = Counter([count for _, count in label_counts.items()])
    print("类别分布情况:")
    for count, num_labels in sorted(count_distribution.most_common(10)):
        print(f"  有 {num_labels} 个类别的样本数为 {count}")
    
    # 查找边界值
    median_count = sorted(label_counts.values())[len(label_counts)//2]
    target_count = min(median_count, 1000)  # 设定目标样本数
    
    # 平衡采样
    balanced_data = []
    class_processed = 0
    rare_classes = 0
    medium_classes = 0
    large_classes = 0
    
    for label, items in sorted(label_groups.items(), key=lambda x: len(x[1])):
        count = len(items)
        class_processed += 1
        
        # 大类下采样 - 随机采样但保证多样性
        if count > max_per_class:
            large_classes += 1
            # 使用分层采样以确保多样性
            if count > max_per_class * 2:  # 非常大的类
                # 对大类进行聚类采样，保留更多的多样性
                intervals = count // max_per_class
                stratified_samples = []
                for i in range(0, count, intervals):
                    end_idx = min(i + intervals, count)
                    samples_from_segment = min(max_per_class // (count // intervals + 1) + 1, end_idx - i)
                    stratified_samples.extend(random.sample(items[i:end_idx], samples_from_segment))
                
                # 如果仍然超过最大数量，随机下采样
                if len(stratified_samples) > max_per_class:
                    balanced_data.extend(random.sample(stratified_samples, max_per_class))
                else:
                    balanced_data.extend(stratified_samples)
            else:
                balanced_data.extend(random.sample(items, max_per_class))
        
        # 中等类别保持原样
        elif min_per_class <= count <= max_per_class:
            medium_classes += 1
            balanced_data.extend(items)
            
        # 小类过采样和数据增强 - 更积极处理
        else:
            rare_classes += 1
            # 基本过采样逻辑
            if count < min_per_class:
                # 先保留所有原始样本
                balanced_data.extend(items)
                
                # 需要增强的样本数 - 更积极地增强（3倍→5倍）
                augment_count = min(min_per_class - count, count * 5)
                
                # 生成增强样本 - 每个原始样本尝试多种增强
                augmented_samples = []
                for _ in range(augment_count):
                    # 随机选择原始样本
                    original = random.choice(items)
                    # 应用多次增强，以创建多样性
                    augmented = augment_classify_sample_advanced(original)
                    augmented_samples.append(augmented)
                
                balanced_data.extend(augmented_samples)
            else:
                balanced_data.extend(items)
        
        # 定期显示进度
        if class_processed % 100 == 0 or class_processed == len(label_groups):
            print(f"已处理 {class_processed}/{len(label_groups)} 个类别 "
                  f"(小类: {rare_classes}, 中类: {medium_classes}, 大类: {large_classes})")
    
    random.shuffle(balanced_data)
    print(f"原始分类数据: {len(classify_data)}条 -> 平衡后: {len(balanced_data)}条")
    
    # 分析平衡后的类别分布
    balanced_counts = {}
    for item in balanced_data:
        target = item['target']
        if target not in balanced_counts:
            balanced_counts[target] = 0
        balanced_counts[target] += 1
    
    # 输出平衡后类别分布的统计信息
    balance_distribution = Counter([count for _, count in balanced_counts.items()])
    print("平衡后的类别分布情况:")
    for count, num_labels in sorted(balance_distribution.most_common(5)):
        print(f"  有 {num_labels} 个类别的样本数为 {count}")
    
    return balanced_data

def augment_classify_sample_advanced(sample):
    """高级版本的分类样本增强 - 应用多种增强策略"""
    # 创建样本副本
    new_item = sample.copy()
    input_text = sample['input']
    
    # 多种增强策略联合使用，提高多样性
    augmented_text = input_text
    
    # 1. 同义词替换（更丰富的同义词库）
    synonyms_dict = {
        "大": ["巨大", "庞大", "宏伟", "伟大", "高大"],
        "小": ["微小", "细小", "迷你", "袖珍", "微型"],
        "好": ["良好", "优秀", "不错", "优良", "优质"],
        "坏": ["糟糕", "恶劣", "不好", "差劲", "劣质"],
        "说": ["讲", "表示", "告诉", "叙述", "陈述"],
        "快": ["迅速", "敏捷", "迅捷", "快速", "快捷"],
        "慢": ["缓慢", "迟缓", "迟钝", "慢吞吞", "拖沓"],
        "美丽": ["漂亮", "好看", "悦目", "动人", "美观"],
        "重要": ["关键", "主要", "核心", "关键性", "至关重要"],
        "公司": ["企业", "集团", "机构", "商行", "厂商"],
        "技术": ["科技", "工艺", "技艺", "专业技能", "工艺流程"],
        "提高": ["增加", "增强", "提升", "增进", "加强"],
        "发展": ["进步", "成长", "壮大", "发达", "前进"],
        "产品": ["商品", "制品", "货品", "出品", "生产品"],
        "市场": ["商场", "销售", "行情", "贸易", "交易"],
        "价格": ["价钱", "价值", "成本", "售价", "金额"],
        "信息": ["消息", "资讯", "情报", "数据", "讯息"],
        "问题": ["疑问", "难题", "困难", "困境", "麻烦"],
        "方法": ["办法", "途径", "手段", "措施", "技巧"],
        "工作": ["职业", "事业", "劳动", "岗位", "职务"],
        "学习": ["研习", "修习", "攻读", "钻研", "求学"],
        "研究": ["研讨", "探究", "探讨", "调查", "考察"],
        "分析": ["解析", "剖析", "研判", "解剖", "推断"],
        "结果": ["成果", "效果", "后果", "产物", "结局"]
    }
    
    # 应用同义词替换 - 更多替换
    for original, replacements in synonyms_dict.items():
        if original in augmented_text:
            # 提高替换概率到80%
            for _ in range(min(3, augmented_text.count(original))):
                if random.random() < 0.8:  # 80%概率替换
                    augmented_text = augmented_text.replace(original, random.choice(replacements), 1)
    
    # 2. 回译效果模拟 - 更复杂的变体
    segmented_text = []
    # 将文本分成3-5个片段
    segment_count = random.randint(3, 5)
    text_length = len(augmented_text)
    for i in range(segment_count):
        start = i * text_length // segment_count
        end = (i + 1) * text_length // segment_count
        segment = augmented_text[start:end]
        
        # 对每个片段应用轻微变换
        if random.random() < 0.7:  # 70%的概率应用变换
            # 交换相邻字符
            if len(segment) > 4 and random.random() < 0.3:
                chars = list(segment)
                swap_position = random.randint(0, len(chars)-2)
                chars[swap_position], chars[swap_position+1] = chars[swap_position+1], chars[swap_position]
                segment = ''.join(chars)
            
            # 随机插入常用助词
            if random.random() < 0.4:
                fillers = ["的", "了", "着", "而", "且", "和", "与", "或"]
                insert_position = random.randint(0, len(segment))
                segment = segment[:insert_position] + random.choice(fillers) + segment[insert_position:]
        
        segmented_text.append(segment)
    
    augmented_text = ''.join(segmented_text)
    
    # 3. 为最终文本添加一些分类特定的提示词，增强类别信息
    class_specific_prompts = [
        "这篇文章主要讲",
        "这是关于",
        "文章描述了",
        "本文介绍",
        "这一内容涉及"
    ]
    
    if random.random() < 0.3:  # 30%概率添加提示词
        prompt = random.choice(class_specific_prompts)
        # 在文本前或后添加提示词
        if random.random() < 0.5:
            augmented_text = prompt + augmented_text
        else:
            augmented_text = augmented_text + "，" + prompt
    
    new_item['input'] = augmented_text
    return new_item

def apply_text_augmentation(text, strategy=None):
    """对文本应用多种增强技术"""
    # 如果没有指定策略，随机选择一种
    if not strategy:
        strategy = random.choice(['synonym', 'insert', 'delete', 'swap', 'back_translation', 'eda'])
    
    words = list(text)
    result = text
    
    if len(words) < 10:
        return text  # 文本太短不增强
    
    if strategy == 'synonym':
        # 同义词替换 - 增加同义词库
        synonyms_dict = {
            "大": ["巨大", "庞大", "宏伟"],
            "小": ["微小", "细小", "迷你"],
            "好": ["良好", "优秀", "不错"],
            "坏": ["糟糕", "恶劣", "不好"],
            "说": ["讲", "表示", "告诉"],
            "快": ["迅速", "敏捷", "迅捷"],
            "慢": ["缓慢", "迟缓", "迟钝"],
            "美丽": ["漂亮", "好看", "悦目"],
            "重要": ["关键", "主要", "核心"],
            "公司": ["企业", "集团", "机构"],
            "技术": ["科技", "工艺", "技艺"],
            "提高": ["增加", "增强", "提升"],
            "发展": ["进步", "成长", "壮大"],
            "产品": ["商品", "制品", "货品"],
            "市场": ["商场", "销售", "行情"],
            "价格": ["价钱", "价值", "成本"]
        }
        
        # 查找文本中可替换的词
        for original, replacements in synonyms_dict.items():
            if original in text:
                # 限制替换次数，不超过3次
                for _ in range(min(3, text.count(original))):
                    if random.random() < 0.7:  # 70% 概率替换
                        result = result.replace(original, random.choice(replacements), 1)
    
    elif strategy == 'insert':
        # 插入填充词
        fillers = ["的", "了", "着", "而", "且", "和", "与", "或", "但", "可以", "这个", "那个", "其实", "确实"]
        
        # 在随机位置插入1-3个填充词
        for _ in range(random.randint(1, 3)):
            pos = random.randint(0, len(words))
            insert_word = random.choice(fillers)
            words.insert(pos, insert_word)
        result = ''.join(words)
    
    elif strategy == 'delete':
        # 删除部分非关键词
        if len(words) > 20:
            # 计算要删除的数量(不超过10%)
            delete_count = min(5, max(1, len(words) // 20))
            
            # 随机选择位置删除
            for _ in range(delete_count):
                if len(words) > 10:  # 确保删除后仍有足够长度
                    pos = random.randint(5, len(words) - 5)  # 避免删除开头和结尾
                    words.pop(pos)
            result = ''.join(words)
    
    elif strategy == 'swap':
        # 交换相邻词
        if len(words) > 10:
            # 交换2-3对相邻字符
            for _ in range(random.randint(2, 3)):
                pos = random.randint(0, len(words) - 2)
                words[pos], words[pos+1] = words[pos+1], words[pos]
            result = ''.join(words)
    
    elif strategy == 'back_translation':
        # 模拟回译效果
        # 实际环境中可使用真实的翻译API，这里用简单模拟
        translation_effects = [
            "将内容进行适当的重组",
            "对句式进行轻微的变化",
            "使用同义表达方式",
            "保持原有意义但改变表述",
        ]
        effect = random.choice(translation_effects)
        
        # 模拟回译效果 - 分段处理长文本
        if len(text) > 50:
            # 将文本分成2-3段
            num_segments = random.randint(2, 3)
            segment_len = len(text) // num_segments
            
            new_segments = []
            for i in range(num_segments):
                start = i * segment_len
                end = start + segment_len if i < num_segments-1 else len(text)
                segment = text[start:end]
                
                # 对每段应用不同的变换
                if random.random() < 0.7:
                    words = list(segment)
                    # 随机交换或替换
                    if len(words) > 5:
                        ops = min(2, len(words) // 10)
                        for _ in range(ops):
                            op_type = random.choice(['swap', 'replace'])
                            if op_type == 'swap' and len(words) > 2:
                                idx1, idx2 = random.sample(range(len(words)), 2)
                                words[idx1], words[idx2] = words[idx2], words[idx1]
                            elif op_type == 'replace' and words:
                                idx = random.randint(0, len(words)-1)
                                words[idx] = random.choice("的地得了是我他她它们这那个和")
                        segment = ''.join(words)
                
                new_segments.append(segment)
            
            result = ''.join(new_segments)
        else:
            # 短文本处理
            result = text
    
    elif strategy == 'eda':
        # 简易版EDA (Easy Data Augmentation)
        # 结合多种操作
        if random.random() < 0.5:
            # 同义词替换
            word_positions = random.sample(range(len(words)), min(3, len(words)//10))
            for pos in word_positions:
                if random.random() < 0.7:
                    words[pos] = random.choice("的地得了是我他她它们这那个和")
        
        if random.random() < 0.3:
            # 随机插入
            for _ in range(min(2, len(words)//15)):
                pos = random.randint(0, len(words))
                insert_word = random.choice("的地得了是我他她它们这那个和")
                words.insert(pos, insert_word)
        
        if random.random() < 0.2 and len(words) > 20:
            # 随机删除
            delete_count = min(2, len(words)//20)
            positions = sorted(random.sample(range(len(words)), delete_count), reverse=True)
            for pos in positions:
                words.pop(pos)
                
        result = ''.join(words)
    
    return result

In [10]:
# 自定义数据集类 - 修改以适应多任务模型
class MultiTaskDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512, pad_idx=0):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.pad_idx = pad_idx
        
        # 按任务类型分类数据
        self.task_data = {
            'classify': [],
            'nli': [],
            'mrc': []
        }
        
        for item in data:
            task_type = item['type']
            if task_type in self.task_data:
                self.task_data[task_type].append(item)
            
        # 统计各任务数据量
        self.task_counts = {task: len(items) for task, items in self.task_data.items()}
        print(f"Task data distribution: {self.task_counts}")
        
        # 为分类任务和NLI任务创建标签映射
        self.label_maps = {}
        self._create_label_maps()
        
        # 识别罕见和常见标签
        self.rare_labels = set()
        self.common_labels = set()
        
        # 针对classify任务统计标签频率
        if self.task_data['classify']:
            try:
                label_counter = Counter([item['target'] for item in self.task_data['classify']])
                total_samples = len(self.task_data['classify'])
                
                # 罕见标签: 出现次数少于总样本0.5%的标签
                threshold_rare = total_samples * 0.005
                # 常见标签: 出现次数多于总样本3%的标签
                threshold_common = total_samples * 0.03
                
                for label, count in label_counter.items():
                    if count < threshold_rare:
                        self.rare_labels.add(label)
                    elif count > threshold_common:
                        self.common_labels.add(label)
                        
                print(f"识别罕见标签{len(self.rare_labels)}个，常见标签{len(self.common_labels)}个")
            except Exception as e:
                print(f"标签统计时出错: {e}")
                # 确保即使出错也能继续运行
                self.rare_labels = set()
                self.common_labels = set()
        
    def _create_label_maps(self):
        """为每个任务创建标签映射 - 修复版本"""
        for task_type in ['classify', 'nli']:
            all_labels = set()
            
            for item in self.task_data[task_type]:
                if 'target' not in item:
                    continue
                    
                # 添加目标到标签集合
                all_labels.add(item['target'])
                
                # 对于NLI任务，确保标签被正确映射
                if task_type == 'nli':
                    # 检查并标准化NLI标签
                    if item['target'] in ['entailment', '蕴含', '是的', 'yes']:
                        item['target'] = 'entailment'
                    elif item['target'] in ['neutral', '中立']:
                        item['target'] = 'neutral'
                    elif item['target'] in ['contradiction', '矛盾', '不是', 'no']:
                        item['target'] = 'contradiction'
            
            # 创建映射字典
            if task_type == 'nli':
                # 对NLI使用标准的三分类标签
                self.label_maps[task_type] = {
                    'entailment': 0,
                    'neutral': 1,
                    'contradiction': 2
                }
                print(f"{task_type} 任务使用标准3分类标签映射")
            else:
                # 对分类任务使用动态映射
                self.label_maps[task_type] = {label: i for i, label in enumerate(sorted(all_labels))}
                print(f"{task_type} 任务有 {len(self.label_maps[task_type])} 个唯一标签")
    
    def __len__(self):
        return sum(self.task_counts.values())
    
    def get_task_item(self, task_type, idx):
        """获取特定任务类型的样本"""
        if idx >= len(self.task_data[task_type]):
            return None
            
        item = self.task_data[task_type][idx]
        
        # 根据任务类型进行不同处理
        if task_type in ['classify', 'nli']:
            return self._process_classification_item(item, task_type)
        elif task_type == 'mrc':
            return self._process_mrc_item(item)
        else:
            return None
    
    def _optimize_answer_choices(self, item):
       target = item['target']
       choices = item.get('answer_choices', [])
       
       # 确保目标在选项中
       if target not in choices:
           choices.append(target)
       
       # 对于选项数量过多的情况，减少选项数量
       if len(choices) > 30:
           # 保留目标选项
           other_choices = [c for c in choices if c != target]
           # 随机选择其他选项，总共不超过30个
           sample_size = min(29, len(other_choices))
           sampled_choices = random.sample(other_choices, sample_size)
           # 确保包含目标选项
           choices = [target] + sampled_choices
           random.shuffle(choices)  # 打乱顺序
       
       return choices
    
    def _process_classification_item(self, item, task_type):
        """处理分类和NLI任务的数据项 - 修复标签映射问题"""
        input_text = item['input']
        target_text = item['target']
        answer_choices = item.get('answer_choices', [])
        
        # 确保选项列表包含目标
        if target_text not in answer_choices and target_text:
            answer_choices.append(target_text)
        
        # 特殊处理NLI任务
        if task_type == 'nli':
            # NLI映射表
            nli_map = {
                'entailment': 0, '蕴含': 0, '是的': 0, 'yes': 0,
                'neutral': 1, '中立': 1,
                'contradiction': 2, '矛盾': 2, '不是': 2, 'no': 2
            }
            
            # 确定标签索引
            if target_text in nli_map:
                label_idx = nli_map[target_text]
            elif len(answer_choices) <= 3:
                # 如果是二/三分类，直接使用位置索引
                label_idx = answer_choices.index(target_text) if target_text in answer_choices else 0
            else:
                # 默认情况
                label_idx = 0
        else:
            # 分类任务处理 - 修复这里的逻辑
            try:
                # 关键修复：不再使用复杂的映射逻辑，直接根据选项位置确定标签
                if target_text in answer_choices:
                    label_idx = answer_choices.index(target_text)
                else:
                    # 找不到目标，使用默认标签
                    label_idx = 0
                    print(f"警告: 目标'{target_text}'不在选项列表中，使用默认标签0")
            except Exception as e:
                print(f"处理分类样本时出错: {e}")
                label_idx = 0
        
        label = torch.tensor(label_idx, dtype=torch.long)
        
        # 编码处理
        encoding = self.tokenizer(
            input_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # 去掉批次维度
        for key in encoding:
            encoding[key] = encoding[key].squeeze(0)
        
        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'token_type_ids': encoding.get('token_type_ids', None),
            'label': label,
            'task_type': task_type,
            'target_text': target_text,
            'answer_choices': answer_choices
        }
        
    def _process_mrc_item(self, item):
        """改进的MRC数据处理函数"""
        input_text = item['input']
        target_text = item['target'].strip()
        
        # 1. 确保有[SEP]分隔符
        if '[SEP]' not in input_text:
            # 将文本划分为上下文和问题
            context = input_text
            question = f"什么是{target_text}？" if target_text and len(target_text) < 10 else "文中的重要信息是什么？"
            input_text = context + " [SEP] " + question
        
        # 2. 分离上下文和问题
        context, question = input_text.split('[SEP]', 1)
        context = context.strip()
        question = question.strip()
        
        # 3. 确保答案在上下文中
        if target_text and target_text not in context:
            # 如果答案不在上下文中，尝试几种策略
            # 策略1: 查找相似片段
            best_overlap = 0
            best_fragment = ""
            
            # 以滑动窗口查找最相似片段
            for i in range(len(context) - len(target_text) + 1):
                fragment = context[i:i+len(target_text)]
                overlap = sum(1 for a, b in zip(fragment, target_text) if a == b)
                if overlap > best_overlap:
                    best_overlap = overlap
                    best_fragment = fragment
            
            # 如果找到相似度足够高的片段，使用它作为新答案
            if best_overlap >= len(target_text) * 0.7:
                target_text = best_fragment
            else:
                # 策略2: 将答案插入到上下文中
                insert_pos = min(len(context) // 2, len(context) - len(target_text))
                context = context[:insert_pos] + target_text + context[insert_pos:]
        
        # 4. 限制上下文长度
        max_context_len = self.max_length - min(100, len(question) + 3) # 预留问题和[CLS][SEP]等标记的空间
        if len(context) > max_context_len:
            # 裁剪策略：保留答案所在区域
            answer_pos = context.find(target_text)
            if answer_pos >= 0:
                half_remain = (max_context_len - len(target_text)) // 2
                start = max(0, answer_pos - half_remain)
                end = min(len(context), answer_pos + len(target_text) + half_remain)
                context = context[start:end]
            else:
                # 如果找不到答案，从开始处截断
                context = context[:max_context_len]
        
        # 重构输入
        input_text = context + " [SEP] " + question
        
        # 5. 编码
        encoding = self.tokenizer(
            input_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt',
            return_offsets_mapping=True
        )
        
        # 提取偏移映射
        offset_mapping = encoding.pop('offset_mapping')[0].numpy()
        
        # 6. 查找答案位置
        start_position, end_position = self._find_answer_position(
            input_text, context, target_text, offset_mapping, encoding['input_ids'][0])
        
        # 确保位置合法
        if end_position < start_position:
            end_position = start_position
        
        # 去掉批次维度
        for key in encoding:
            if isinstance(encoding[key], torch.Tensor):
                encoding[key] = encoding[key].squeeze(0)
        
        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'token_type_ids': encoding.get('token_type_ids', torch.zeros_like(encoding['input_ids'])),
            'start_position': start_position,
            'end_position': end_position,
            'task_type': 'mrc',
            'target_text': target_text
        }
        
    def _find_answer_position(self, full_text, context, answer, offset_mapping, input_ids):
        """改进版的答案位置查找"""
        # 方法1: 直接查找答案在原文中的位置
        context_offset = full_text.find(context)
        answer_start = context.find(answer)
        
        if answer_start != -1:
            answer_start += context_offset
            answer_end = answer_start + len(answer) - 1
            
            # 找到对应的token位置
            start_position = end_position = None
            
            for idx, (start_offset, end_offset) in enumerate(offset_mapping):
                if start_offset <= answer_start < end_offset:
                    start_position = idx
                
                if start_offset <= answer_end < end_offset:
                    end_position = idx
                    break
            
            if start_position is not None and end_position is not None:
                return torch.tensor(start_position), torch.tensor(end_position)
        
        # 方法2: 使用最长公共子序列查找
        answer_tokens = self.tokenizer.encode(answer, add_special_tokens=False)
        input_tokens = input_ids.tolist()
        
        max_overlap = 0
        best_start = 1  # 默认跳过[CLS]
        best_end = 1
        
        # 在输入序列中寻找最长的公共子序列
        for i in range(1, len(input_tokens) - len(answer_tokens) + 1):
            overlap = self._compute_lcs(input_tokens[i:i+len(answer_tokens)], answer_tokens)
            if overlap > max_overlap:
                max_overlap = overlap
                best_start = i
                best_end = i + len(answer_tokens) - 1
        
        # 如果找到了足够好的匹配
        if max_overlap > len(answer_tokens) * 0.5:
            return torch.tensor(best_start), torch.tensor(best_end)
        
        # 备选方案: 返回标题后的位置作为默认值
        return torch.tensor(1), torch.tensor(min(5, len(input_ids)-1))

    def _compute_lcs(self, seq1, seq2):
        """计算最长公共子序列长度"""
        m, n = len(seq1), len(seq2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if seq1[i-1] == seq2[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        
        return dp[m][n]
    
    def _augment_mrc(self, input_text, target_text):
        """增强版MRC数据增强函数"""
        # 将输入文本分为上下文和问题部分
        if '[SEP]' in input_text:
            context, question = input_text.split('[SEP]', 1)
            context = context.strip()
            question = question.strip()
        else:
            context = input_text
            question = ""
        
        # 查找答案在上下文中的位置
        start_idx = context.find(target_text)
        if start_idx == -1:
            return input_text, target_text
        
        # 不同增强策略的权重 - 可以根据需要调整
        aug_weights = {
            'no_change': 0.2,          # 不改变
            'add_affixes': 0.2,        # 添加前后缀
            'character_edit': 0.15,    # 字符编辑
            'context_noise': 0.2,      # 上下文噪声
            'question_noise': 0.15,    # 问题噪声
            'synonym_replace': 0.1,    # 同义词替换 - 新增
        }
        
        # 按权重选择增强策略
        aug_strategy = random.choices(
            list(aug_weights.keys()),
            weights=list(aug_weights.values()),
            k=1
        )[0]
        
        # 不做改变
        if aug_strategy == 'no_change':
            return input_text, target_text
        
        # 添加前后缀
        elif aug_strategy == 'add_affixes':
            # 60%概率添加后缀，40%概率添加前缀
            if random.random() < 0.6:
                suffix = random.choice(["。", "，", "的", "了", "是", "地"])
                new_target = target_text + suffix
                new_context = context[:start_idx] + new_target + context[start_idx+len(target_text):]
            else:
                prefix = random.choice(["那个", "这个", "有个", "是", "就是"])
                new_target = prefix + target_text
                new_context = context[:start_idx] + new_target + context[start_idx+len(target_text):]
        
        # 字符编辑
        elif aug_strategy == 'character_edit':
            chars = list(target_text)
            
            if len(chars) <= 2:
                # 对于短答案，只添加字符
                pos = random.randint(0, len(chars))
                chars.insert(pos, random.choice("的地得了是我他她它们"))
            else:
                # 随机选择编辑操作：替换、删除或插入
                edit_op = random.choice(['replace', 'delete', 'insert'])
                
                if edit_op == 'replace' and len(chars) > 1:
                    pos = random.randint(0, len(chars) - 1)
                    chars[pos] = random.choice("的地得了是我他她它们这那和与")
                elif edit_op == 'delete' and len(chars) > 3:
                    pos = random.randint(0, len(chars) - 1)
                    chars.pop(pos)
                else:  # insert
                    pos = random.randint(0, len(chars))
                    chars.insert(pos, random.choice("的地得了是我他她它们"))
            
            new_target = ''.join(chars)
            new_context = context[:start_idx] + new_target + context[start_idx+len(target_text):]
        
        # 上下文噪声
        elif aug_strategy == 'context_noise':
            # 在答案周围添加噪声
            context_chars = list(context)
            
            # 答案前的区域
            if start_idx > 5:
                noise_pos = random.randint(max(0, start_idx - 10), start_idx - 1)
                if random.random() < 0.7:  # 替换
                    context_chars[noise_pos] = random.choice("的地得了是我他她它们这那和与")
                else:  # 插入
                    context_chars.insert(noise_pos, random.choice("的地得了是我他她它们"))
            
            # 答案后的区域
            end_idx = start_idx + len(target_text)
            if end_idx < len(context) - 5:
                noise_pos = random.randint(end_idx + 1, min(end_idx + 10, len(context) - 1))
                if random.random() < 0.7:  # 替换
                    context_chars[noise_pos] = random.choice("的地得了是我他她它们这那和与")
                else:  # 插入
                    context_chars.insert(noise_pos, random.choice("的地得了是我他她它们"))
            
            new_context = ''.join(context_chars)
            # 重新计算答案位置
            start_idx = new_context.find(target_text)
            if start_idx == -1:  # 如果找不到原答案，保持不变
                new_context = context
            new_target = target_text
        
        # 问题噪声
        elif aug_strategy == 'question_noise':
            new_context = context
            new_target = target_text
            
            if question:
                q_chars = list(question)
                
                # 应用多种问题变换
                q_ops = random.choices(['swap', 'replace', 'add', 'remove'], k=min(2, len(q_chars)))
                
                for op in q_ops:
                    if op == 'swap' and len(q_chars) >= 2:
                        # 交换相邻字符
                        pos = random.randint(0, len(q_chars) - 2)
                        q_chars[pos], q_chars[pos + 1] = q_chars[pos + 1], q_chars[pos]
                    elif op == 'replace' and q_chars:
                        # 替换一个字符
                        pos = random.randint(0, len(q_chars) - 1)
                        q_chars[pos] = random.choice("的地得了是我他她它们什么怎么为什么谁哪里")
                    elif op == 'add':
                        # 添加一个字符
                        pos = random.randint(0, len(q_chars))
                        q_chars.insert(pos, random.choice("的地得了是什么怎么谁哪里"))
                    elif op == 'remove' and len(q_chars) > 3:
                        # 移除一个字符
                        pos = random.randint(0, len(q_chars) - 1)
                        q_chars.pop(pos)
                
                question = ''.join(q_chars)
        
        if aug_strategy == 'synonym_replace':
            # 同义词字典 - 实际应用时可以更加丰富
            synonym_dict = {
                "说": ["讲", "表示", "告诉"],
                "看": ["瞧", "望", "观察"],
                "大": ["巨大", "庞大", "宏伟"],
                "小": ["微小", "细小", "迷你"],
                "好": ["良好", "优秀", "不错"],
                "坏": ["糟糕", "恶劣", "不好"],
                # 可以添加更多同义词
            }
            
            # 替换上下文中的一些词
            context_words = list(context)
            for word, synonyms in synonym_dict.items():
                if word in context:
                    # 不替换答案中的词
                    safe_context = context.replace(target_text, "✂"*len(target_text))
                    for idx in [i for i, char in enumerate(safe_context) if char == word]:
                        if random.random() < 0.5:  # 50%概率替换
                            context_words[idx] = random.choice(synonyms)
            
            new_context = ''.join(context_words)
            new_target = target_text  # 答案保持不变
            
            # 重建输入文本
            if question:
                new_input = new_context + " [SEP] " + question
            else:
                new_input = new_context
            
            return new_input, new_target
        
        # 重建输入文本
        if question:
            new_input = new_context + " [SEP] " + question
        else:
            new_input = new_context
        
        return new_input, new_target
    
    def enhance_mrc_data(self, input_text, target_text):
        """增强MRC输入和目标文本"""
        # 确保目标文本在输入中能够找到
        if input_text.find(target_text) == -1:
            # 原始文本中找不到答案，尝试将答案添加到输入中
            if '[SEP]' in input_text:
                context, question = input_text.split('[SEP]', 1)
                # 在上下文中适当位置插入答案
                insertion_point = len(context) // 2
                context = context[:insertion_point] + target_text + context[insertion_point:]
                input_text = context + '[SEP]' + question
        
        return input_text

    def _replace_char_in_answer(self, text, context, start_idx):
        """替换答案中的一个字符"""
        if len(text) <= 3:
            return text, context
        
        # 选择要替换的位置，避免首尾
        pos = random.randint(1, len(text)-2)
        # 选择一个可能的替换字符
        replacement = random.choice("的一是在有和人这中大为上个国我以要他时来用们生到作地于出就分对成会可主发年动同工也能下过子说产种面而方后多定行学法所民得经十三之进着等部度家电力里如水化高自二理起小物现实加量都两体制机当使点从业本去把性好应开它合还因由其些然前外天政四日那社义事平形相全表间样与关各重新线内数正心反你明看原又么利比或但质气第向道命此变条只没结解问意建月公无系军很情者最立代想已通并提直题党程展五果料象员革位入常文总次品式活设及管特件长求老头基资边流路级少图山统接知较将组见计别她手角期根论运农指几九区强放决西被干做必战先回则任取据处队南给色光门即保治北造百规热领七海口东导器压志世金增争济阶油思术极交受联什认六共权收证改清己美再采转更单风切打白教速花带安场身车例真务具万每目至达走积示议声报斗完类八离华名确才科张信马节话米整空元况今集温传土许步群广石记需段研界拉林律叫且究观越织装影算低持音众书布复容儿须际商非验连断深难近矿千周委素技备半办青省列习响约支般史感劳便团往酸历市克何除消构府称太准精值号率族维划选标写存候毛亲快效斯院查江型眼王按格养易置派层片始却专状育厂京识适属圆包火住调满县局照参红细引听该铁价严龙飞") 
        
        new_text = text[:pos] + replacement + text[pos+1:]
        new_context = context[:start_idx] + new_text + context[start_idx+len(text):]
        
        return new_text, new_context

    def _add_noise_to_question(self, context, question):
        """为问题添加噪声，但保留上下文不变"""
        words = list(question)
        # 随机替换、删除或插入字符
        for i in range(min(2, len(words))):
            pos = random.randint(0, len(words) - 1)
            if random.random() < 0.7:  # 替换
                words[pos] = random.choice("的地得了吗呢啊哦嗯呀哈你我他她它们为什么怎谁")
            else:  # 删除
                if len(words) > 5:  # 确保不会删除太多
                    words.pop(pos)
        
        noised_question = ''.join(words)
        
        # 重建输入文本，保留上下文不变
        return context + " [SEP] " + noised_question
    
    def _augment_classify_text(self, text):
        """针对classify任务优化的文本增强函数"""
        # 提取问题和内容
        question_part = ""
        content_part = text
        
        # 常见问题前缀
        question_prefixes = ["这是关于哪方面的新闻", "哪个类别最好的描述了", "这篇新闻会出现在哪个栏目", 
                        "这段话属于什么类别", "阅读下列论文摘要", "下面两个句子语义是",
                        "以下两句话的意思相同"]
        
        # 尝试分离问题和内容
        for prefix in question_prefixes:
            if prefix in text[:50]:  # 只查找开头部分
                parts = text.split("？", 1)  # 在第一个问号处分割
                if len(parts) > 1:
                    question_part = parts[0] + "？"
                    content_part = parts[1]
                    break
        
        # 如果内容部分超过150个字符，才考虑增强
        if len(content_part) > 150:
            # 降低增强概率和强度
            if random.random() < 0.15:  # 降低增强概率
                augmented_content = self._augment_content(content_part, light=True)  # 使用轻度增强
                return question_part + augmented_content
        
        # 对于较短文本或问题部分重要的文本，不进行增强
        return text

    def _augment_content(self, content, light=False):
        """内容增强函数，light参数控制增强强度"""
        words = list(content)
        if not words:
            return content
                
        # 轻微增强：只进行少量修改
        if light:
            max_changes = max(1, min(2, len(words) // 20))  # 确保至少为1，最多为2
        else:
            max_changes = max(1, min(5, len(words) // 10))  # 确保至少为1，最多为5
        
        # 保证changes至少为1，不超过max_changes
        changes = min(max_changes, max(1, len(words) // 30))  # 默认每30个字符改1处
        
        # 保护特殊符号和数字
        def is_protected(char):
            return char.isdigit() or char in ",.。，、；：''""《》【】！？"
        
        # 找出可以修改的位置
        valid_positions = [i for i, char in enumerate(words) if not is_protected(char)]
        
        # 如果没有可修改的位置或内容太短，直接返回原文本
        if not valid_positions or len(words) <= 3:
            return content
                
        # 随机选择位置进行修改
        for _ in range(min(changes, len(valid_positions))):  # 确保不超过有效位置数量
            if not valid_positions:
                break
                    
            op = random.choice(['replace', 'delete', 'swap'])
            
            if op == 'replace' and valid_positions:
                pos = random.choice(valid_positions)
                words[pos] = random.choice("的地得了吗呢啊哦嗯呀哈")
                valid_positions.remove(pos)
                
            elif op == 'delete' and len(words) > 20 and valid_positions:
                pos = random.choice(valid_positions)
                words.pop(pos)
                # 更新可用位置
                valid_positions = [p if p < pos else p-1 for p in valid_positions if p != pos]
                
            elif op == 'swap' and len(valid_positions) >= 2:
                pos1, pos2 = random.sample(valid_positions, 2)
                words[pos1], words[pos2] = words[pos2], words[pos1]
                if pos1 in valid_positions: valid_positions.remove(pos1)
                if pos2 in valid_positions: valid_positions.remove(pos2)
        
        return ''.join(words)
        
    def __getitem__(self, idx):
        """根据索引获取样本，但按任务类型分配索引"""
        # 计算每个任务的样本量比例
        total = self.__len__()
        task_probs = {task: count/total for task, count in self.task_counts.items()}
        
        # 根据比例随机选择任务类型
        task_type = random.choices(
            list(task_probs.keys()),
            weights=list(task_probs.values()),
            k=1
        )[0]
        
        # 为选中的任务类型随机选择一个索引
        task_idx = random.randint(0, self.task_counts[task_type] - 1)
        
        try:
            return self.get_task_item(task_type, task_idx)
        except Exception as e:
            print(f"Error processing {task_type} item at index {task_idx}: {e}")
            # 返回一个默认项避免批处理失败
            return self.get_task_item('classify', 0) or None

## 3.2 数据收集

In [11]:
def custom_collate_fn(batch):
    """处理不同大小的批次数据"""
    # 过滤掉None值
    batch = [item for item in batch if item is not None]
    
    if len(batch) == 0:
        return {}
    
    # 首先收集所有样本中出现的键
    all_keys = set()
    for item in batch:
        all_keys.update(item.keys())
    
    result = {}
    for key in all_keys:
        # 只处理在所有样本中都存在的键，或特殊处理某些键
        if all(key in item for item in batch):
            # 对于字符串或列表类型的字段
            if key in ['task_type', 'target_text', 'answer_choices']:
                result[key] = [item[key] for item in batch]
            # 对于张量类型
            elif isinstance(batch[0][key], torch.Tensor):
                shapes = [item[key].shape for item in batch]
                if all(shape == shapes[0] for shape in shapes):
                    # 如果所有形状一致，则进行常规的堆叠
                    result[key] = torch.stack([item[key] for item in batch])
                else:
                    # 如果形状不一致，则进行填充
                    max_len = max(shape[0] for shape in shapes)
                    padded_tensors = []
                    for item in batch:
                        tensor = item[key]
                        if tensor.shape[0] < max_len:
                            padding = torch.zeros(max_len - tensor.shape[0], *tensor.shape[1:], 
                                                dtype=tensor.dtype, device=tensor.device)
                            tensor = torch.cat([tensor, padding], dim=0)
                        padded_tensors.append(tensor)
                    result[key] = torch.stack(padded_tensors)
        else:
            # 特殊处理MRC任务特有的字段
            if key in ['start_position', 'end_position']:
                # 找出拥有该键的样本索引
                indices = [i for i, item in enumerate(batch) if key in item]
                if indices:
                    # 创建包含有效值的列表
                    values = [batch[i][key] for i in indices]
                    # 记录这些样本的索引，以便训练时使用
                    result[f'{key}_indices'] = indices
                    result[key] = torch.stack(values)
    
    return result

## 3.3 数据加载器创建

In [12]:
# 数据加载器创建函数
def create_dataloaders(train_data, val_data, tokenizer, batch_size=8):
    train_dataset = MultiTaskDataset(train_data, tokenizer)
    val_dataset = MultiTaskDataset(val_data, tokenizer)
    
    # 使用num_workers加速数据加载
    num_workers = min(4, os.cpu_count())
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=num_workers,
        pin_memory=True,  # GPU加速
        collate_fn=custom_collate_fn  # 使用自定义的collate函数
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,  # GPU加速
        collate_fn=custom_collate_fn  # 使用自定义的collate函数
    )
    
    return train_loader, val_loader, train_dataset.label_map

## 3.4 任务特定数据加载器

In [13]:
def create_task_specific_dataloaders(train_data, val_data, tokenizer, batch_size=8):
    """创建按任务类型分类的数据加载器"""
    # 分离classify数据并应用平衡处理
    train_classify_data = [item for item in train_data if item['type'] == 'classify']
    train_other_data = [item for item in train_data if item['type'] != 'classify']
    
    # 应用增强的平衡处理
    balanced_classify_data = balance_classify_data_improved(train_classify_data)
    
    # 重新组合数据
    balanced_train_data = balanced_classify_data + train_other_data
    
    train_dataset = MultiTaskDataset(train_data, tokenizer)
    val_dataset = MultiTaskDataset(val_data, tokenizer)
    
    # 使用num_workers加速数据加载
    num_workers = min(4, os.cpu_count())
    
    # 创建任务特定的数据加载器
    train_loaders = {}
    val_loaders = {}
    
    for task_type in ['classify', 'nli', 'mrc']:
        if train_dataset.task_counts[task_type] > 0:
            # 创建任务特定的数据集包装器
            train_task_dataset = TaskSpecificDataset(train_dataset, task_type)
            train_loaders[task_type] = DataLoader(
                train_task_dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=num_workers,
                pin_memory=True,
                collate_fn=lambda x: task_collate_fn(x, task_type)
            )
        
        if val_dataset.task_counts[task_type] > 0:
            val_task_dataset = TaskSpecificDataset(val_dataset, task_type)
            val_loaders[task_type] = DataLoader(
                val_task_dataset,
                batch_size=8,
                num_workers=num_workers,
                pin_memory=True,
                collate_fn=lambda x: task_collate_fn(x, task_type)
            )
    
    return train_loaders, val_loaders, train_dataset.label_maps

class TaskSpecificDataset(Dataset):
    """任务特定的数据集包装器"""
    def __init__(self, multi_task_dataset, task_type):
        self.dataset = multi_task_dataset
        self.task_type = task_type
        self.indices = list(range(self.dataset.task_counts[task_type]))
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        return self.dataset.get_task_item(self.task_type, self.indices[idx])

# 创建任务特定的数据加载器
def task_collate_fn(batch, task_type):
    """任务特定的批处理函数"""
    # 过滤掉None值
    batch = [item for item in batch if item is not None]
    
    if len(batch) == 0:
        return {}
    
    result = {}
    
    # 常规键处理
    for key in batch[0].keys():
        if key in ['task_type', 'target_text', 'answer_choices']:
            result[key] = [item[key] for item in batch]
        elif isinstance(batch[0][key], torch.Tensor):
            result[key] = torch.stack([item[key] for item in batch])
    
    # MRC任务特殊处理
    if task_type == 'mrc':
        if all('start_position' in item and 'end_position' in item for item in batch):
            result['start_position'] = torch.stack([item['start_position'] for item in batch])
            result['end_position'] = torch.stack([item['end_position'] for item in batch])
    
    return result

# 4 定义自定义损失函数和模型

## 4.1 损失函数

In [14]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing
        
    def forward(self, x, target):
        confidence = 1. - self.smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

In [15]:
def weighted_cross_entropy_loss(outputs, labels, class_weights=None):
    """带类别权重的交叉熵损失函数"""
    if class_weights is not None:
        num_classes = outputs.size(-1)  # 获取模型输出的类别数量
        
        # 确保所有类别都有权重
        if len(class_weights) < num_classes:
            # 扩展权重列表以覆盖所有类别
            additional_weights = [1.0] * (num_classes - len(class_weights))
            extended_weights = class_weights + additional_weights
            weight_tensor = torch.tensor(extended_weights, device=labels.device)
        else:
            # 只使用需要的权重数量
            weight_tensor = torch.tensor(class_weights[:num_classes], device=labels.device)
        
        return F.cross_entropy(outputs, labels, weight=weight_tensor)
    else:
        return F.cross_entropy(outputs, labels)

In [16]:
def focal_cross_entropy_loss(logits, target, gamma=2.0):
    """Focal Loss for MRC任务，降低简单样本权重，关注难样本"""
    ce_loss = F.cross_entropy(logits, target, reduction='none')
    p = torch.exp(-ce_loss)
    loss = (1 - p) ** gamma * ce_loss
    return loss.mean()

def span_constraint_loss(start_logits, end_logits):
    """添加跨度约束损失，确保start_pos <= end_pos"""
    batch_size, seq_length = start_logits.size()
    
    # 创建位置矩阵
    start_pos = torch.arange(seq_length).expand(batch_size, seq_length).to(start_logits.device)
    end_pos = torch.arange(seq_length).expand(batch_size, seq_length).to(end_logits.device)
    
    # 计算所有可能的start-end对的分数
    start_scores = F.softmax(start_logits, dim=-1).unsqueeze(2)  # [B, S, 1]
    end_scores = F.softmax(end_logits, dim=-1).unsqueeze(1)      # [B, 1, S]
    
    # 组合分数 [B, S, S]
    span_scores = start_scores * end_scores
    
    # 创建掩码，对于所有 start_pos > end_pos 的位置为1
    invalid_span_mask = (start_pos.unsqueeze(2) > end_pos.unsqueeze(1))
    
    # 对无效跨度惩罚
    invalid_span_scores = span_scores.masked_fill(~invalid_span_mask, 0.0)
    constraint_loss = invalid_span_scores.sum(dim=(1, 2)).mean()
    
    return constraint_loss

In [17]:
def compute_mrc_loss(start_logits, end_logits, start_positions, end_positions, answer_texts=None, input_ids=None, tokenizer=None):
    """增强的MRC损失计算函数，处理多重情况"""
    # 常规步骤: 计算开始和结束位置的交叉熵损失
    start_loss = F.cross_entropy(start_logits, start_positions)
    end_loss = F.cross_entropy(end_logits, end_positions)
    
    # 添加位置约束损失
    batch_size, seq_length = start_logits.size()
    
    # 创建可能的开始/结束对的矩阵表示
    start_pos = torch.arange(seq_length).expand(batch_size, seq_length).to(start_logits.device)
    end_pos = torch.arange(seq_length).expand(batch_size, seq_length).to(end_logits.device)
    
    # 阻止end_pos < start_pos的情况
    start_scores = F.softmax(start_logits, dim=1).unsqueeze(2)  # [B, S, 1]
    end_scores = F.softmax(end_logits, dim=1).unsqueeze(1)      # [B, 1, S]
    pairwise_scores = start_scores * end_scores                 # [B, S, S]
    
    # 创建掩码，对于所有无效跨度(end < start)标记为True
    invalid_pairs_mask = end_pos.unsqueeze(1) < start_pos.unsqueeze(2)  # [B, S, S]
    
    # 惩罚无效跨度
    invalid_scores = pairwise_scores.masked_fill(~invalid_pairs_mask, 0.0)
    span_constraint_loss = invalid_scores.sum() / batch_size
    
    # 限制答案长度损失
    if input_ids is not None and tokenizer is not None and answer_texts is not None:
        # 鼓励合理长度的答案
        length_penalties = []
        
        for b in range(batch_size):
            pred_start = torch.argmax(start_logits[b]).item()
            pred_end = torch.argmax(end_logits[b]).item()
            
            if pred_end < pred_start:
                pred_end = pred_start
                
            pred_length = pred_end - pred_start + 1
            target_length = end_positions[b] - start_positions[b] + 1
            
            # 如果预测长度偏离目标长度太多，则惩罚
            length_diff = abs(pred_length - target_length)
            length_penalty = torch.tensor(min(length_diff / 10.0, 1.0), 
                                        device=start_logits.device)
            length_penalties.append(length_penalty)
            
        if length_penalties:
            length_loss = torch.stack(length_penalties).mean() * 0.5
        else:
            length_loss = torch.tensor(0.0, device=start_logits.device)
    else:
        length_loss = torch.tensor(0.0, device=start_logits.device)
        
    # 组合所有损失
    total_loss = (start_loss + end_loss) / 2 + span_constraint_loss * 0.2 + length_loss
    
    return total_loss

In [18]:
def focal_loss(outputs, labels, gamma=2.0, alpha=0.25):
    """处理类别不平衡的Focal Loss"""
    ce_loss = F.cross_entropy(outputs, labels, reduction='none')
    p_t = torch.exp(-ce_loss)
    loss = (1 - p_t) ** gamma * ce_loss
    return loss.mean()

## 4.2 模型定义

In [19]:
class SimplifiedMRCHead(nn.Module):
    """内存优化的MRC头部结构"""
    def __init__(self, hidden_size):
        super(SimplifiedMRCHead, self).__init__()
        
        # 用更轻量的单层LSTM替代多层LSTM
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size // 2,
            num_layers=1,  # 从3层减少到1层
            bidirectional=True,
            batch_first=True,
            dropout=0.1
        )
        
        # 精简注意力层
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=4,  # 从8减少到4
            dropout=0.1
        )
        
        # 输出层
        self.qa_outputs = nn.Linear(hidden_size, 2)
    
    def forward(self, sequence_output, attention_mask=None, token_type_ids=None):
        # LSTM增强上下文依赖
        lstm_output, _ = self.lstm(sequence_output)
        
        # 掩码处理
        key_padding_mask = None
        if attention_mask is not None:
            key_padding_mask = (1 - attention_mask).bool()
            
        # 自注意力机制
        attn_output, _ = self.attention(
            lstm_output.transpose(0, 1),
            lstm_output.transpose(0, 1),
            lstm_output.transpose(0, 1),
            key_padding_mask=key_padding_mask
        )
        attn_output = attn_output.transpose(0, 1)
        
        # 残差连接
        enhanced_output = lstm_output + attn_output
        
        # 预测开始和结束位置
        logits = self.qa_outputs(enhanced_output)
        
        # 分割出开始和结束logits
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        
        # 应用注意力掩码
        if attention_mask is not None:
            start_logits = start_logits + (1 - attention_mask) * -10000.0
            end_logits = end_logits + (1 - attention_mask) * -10000.0
            
        return start_logits, end_logits

In [20]:
class ClassifyHead(nn.Module):
    """简化且有效的分类头部"""
    def __init__(self, hidden_size, num_labels, dropout_prob=0.1):
        super(ClassifyHead, self).__init__()
        
        # 简化特征提取网络
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.GELU()
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        
        # 分类层
        self.classifier = nn.Linear(hidden_size, num_labels)
        
        # 初始化
        self._init_weights()
    
    def _init_weights(self):
        # 精心设计的初始化方式，避免梯度消失/爆炸
        nn.init.xavier_uniform_(self.dense.weight)
        nn.init.constant_(self.dense.bias, 0)
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.constant_(self.classifier.bias, 0)
        
    def forward(self, hidden_states, attention_mask=None, pooled_output=None):
        # 使用[CLS]表示或平均池化
        if pooled_output is not None:
            # 使用BERT的pooled_output (CLS token)
            x = pooled_output
        else:
            # 使用序列的平均表示
            if attention_mask is not None:
                # 有效token的平均值，忽略padding
                mask_expanded = attention_mask.unsqueeze(-1).float()
                sum_embeddings = torch.sum(hidden_states * mask_expanded, 1)
                sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
                x = sum_embeddings / sum_mask
            else:
                # 简单平均
                x = torch.mean(hidden_states, dim=1)
        
        # 应用层
        x = self.dense(x)
        x = self.activation(x)
        x = self.layer_norm(x)
        x = self.dropout(x)
        logits = self.classifier(x)
        
        return logits

In [21]:
# 使用预训练的BERT替代自定义编码器
class MultitaskBertModel(nn.Module):
    def __init__(self, num_labels=2):
        super(MultitaskBertModel, self).__init__()
        
        # 加载预训练的BERT模型
        self.bert = AutoModel.from_pretrained("./chinese-roberta-wwm-ext")
        
        # 使用简化的分类头
        self.classify_head = ClassifyHead(self.bert.config.hidden_size, num_labels)
        self.nli_head = nn.Linear(self.bert.config.hidden_size, num_labels)
        
        # 使用简化的MRC头部
        self.mrc_head = SimplifiedMRCHead(self.bert.config.hidden_size)
        
        # 初始化权重
        self._init_weights()
        
        # 解冻所有层，让模型充分训练
        for param in self.bert.parameters():
            param.requires_grad = True
    
    def _init_weights(self):
        # NLI头初始化
        nn.init.xavier_uniform_(self.nli_head.weight)
        nn.init.zeros_(self.nli_head.bias)
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, task_type=None):
        # 处理多样本批次中可能有不同任务类型的情况
        if isinstance(task_type, list):
            # 批次中有多个样本，可能是不同任务类型
            outputs = []
            for i in range(input_ids.size(0)):
                sample_input_ids = input_ids[i:i+1]
                sample_attention_mask = attention_mask[i:i+1] if attention_mask is not None else None
                sample_token_type_ids = token_type_ids[i:i+1] if token_type_ids is not None else None
                sample_task_type = task_type[i]
                
                # 调用单样本处理
                sample_output = self.forward_single(
                    sample_input_ids, 
                    sample_attention_mask,
                    sample_token_type_ids,
                    sample_task_type
                )
                outputs.append(sample_output)
            
            # 根据任务类型合并输出
            return torch.cat(outputs, dim=0)
        else:
            # 单一任务类型的批次，直接处理
            return self.forward_single(input_ids, attention_mask, token_type_ids, task_type)
        
    def forward_single(self, input_ids, attention_mask=None, token_type_ids=None, task_type=None):
        # 使用BERT提取特征
        outputs = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids
        )
        
        # 获取序列表示和[CLS]的表示
        sequence_output = outputs.last_hidden_state
        pooled_output = outputs.pooler_output
        
        if task_type == 'mrc':
            # MRC任务
            return self.mrc_head(sequence_output, attention_mask, token_type_ids)
        
        elif task_type == 'classify':
            # 分类任务
            return self.classify_head(sequence_output, attention_mask, pooled_output)
        
        elif task_type == 'nli':
            # NLI任务
            return self.nli_head(pooled_output)
        
        else:
            # 默认使用分类头
            return self.classify_head(sequence_output, attention_mask, pooled_output)
    
    def init_classify_head_properly(self):
        """正确初始化分类头权重，解决冷启动问题"""
        # 分类头部分
        if hasattr(self.classify_head, 'dense'):
            nn.init.normal_(self.classify_head.dense.weight, std=0.02)
            nn.init.zeros_(self.classify_head.dense.bias)
        
        if hasattr(self.classify_head, 'classifier'):
            nn.init.normal_(self.classify_head.classifier.weight, std=0.02)
            nn.init.zeros_(self.classify_head.classifier.bias)

# 5 训练和评估函数

## 5.1 训练函数

In [22]:
# 训练函数 - 优化以适用于GPU
def train(model, train_loader, val_loader, optimizer, scheduler, device, best_model_path, tokenizer, num_epochs=3, eval_steps=100):
    def focal_loss(outputs, labels, gamma=2.0, alpha=0.25):
       """专门处理类别不平衡的Focal Loss"""
       ce_loss = F.cross_entropy(outputs, labels, reduction='none')
       p_t = torch.exp(-ce_loss)
       loss = (1 - p_t) ** gamma * ce_loss
       return loss.mean()
    
    best_val_score = 0
    global_step = 0
    scaler = torch.cuda.amp.GradScaler()  # 使用混合精度训练
    patience = 3  # 早停的耐心参数
    early_stopping_counter = 0  # 早停计数器
    
    # 用于Kaggle输出的结果记录
    results_history = []
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, batch in enumerate(progress_bar):
            # 检查批次是否为空
            if not batch or len(batch) == 0:
                print("跳过空批次")
                continue
                
            # 检查输入IDs是否存在
            if 'input_ids' not in batch:
                print(f"批次 {batch_idx} 中没有 input_ids，跳过")
                continue
        
        for batch_idx, batch in enumerate(progress_bar):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch.get('attention_mask', None)
            if attention_mask is not None:
                attention_mask = attention_mask.to(device)
            src_padding_mask = batch.get('src_padding_mask', None)
            if src_padding_mask is not None:
                src_padding_mask = src_padding_mask.to(device)
            labels = batch['label'].to(device)
            task_types = batch['task_type']
            
            optimizer.zero_grad()
            
            # 仅处理有效样本
            valid_indices = (labels != -100).nonzero(as_tuple=True)[0]
            
            if len(valid_indices) > 0:
                batch_input_ids = input_ids[valid_indices]
                batch_attention_mask = attention_mask[valid_indices] if attention_mask is not None else None
                batch_src_padding_mask = src_padding_mask[valid_indices] if src_padding_mask is not None else None
                batch_labels = labels[valid_indices]
                batch_task_types = [task_types[i] for i in valid_indices]
                
                # 使用混合精度训练
                with torch.cuda.amp.autocast():
                    # 获取token_type_ids并移至设备
                    token_type_ids = batch.get('token_type_ids', None)
                    if token_type_ids is not None:
                        # 先将索引移到与token_type_ids相同的设备上 (CPU)
                        cpu_valid_indices = valid_indices.cpu()
                        # 使用CPU上的索引切片CPU上的张量
                        token_type_ids = token_type_ids[cpu_valid_indices]
                        # 然后将结果移到目标设备
                        token_type_ids = token_type_ids.to(device)
                        
                    outputs = model(
                        input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        token_type_ids=token_type_ids,
                        task_type=batch_task_types
                    )
                    
                    # 分开处理不同任务类型
                    task_weights = {
                        'classify': 1.0,
                        'nli': 2.0,      # 增加NLI任务权重
                        'mrc': 3.0       # 进一步增加MRC任务权重
                    }
                    
                    # 初始化每种任务的损失
                    losses = {}
                    task_counts = {}
                    
                    # 按任务类型分组计算损失
                    for task_type in set(batch_task_types):
                        task_indices = [i for i, t in enumerate(batch_task_types) if t == task_type]
                        task_counts[task_type] = len(task_indices)
                        
                        if task_type in ['classify', 'nli'] and task_indices:
                            task_outputs = outputs[task_indices]
                            task_labels = batch_labels[task_indices]
                            
                            # 使用标签平滑交叉熵来提高泛化能力
                            if task_type == 'nli':
                                # NLI任务可能需要更多正则化
                                loss_fn = LabelSmoothingCrossEntropy(smoothing=0.1)
                            elif task_type == 'classify':
                                # 对classify任务使用focal_loss
                                losses[task_type] = focal_loss(task_outputs, task_labels) * task_weights[task_type]
                                continue
                            else:
                                loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
                                
                            losses[task_type] = loss_fn(task_outputs, task_labels) * task_weights[task_type]

                    # 在train函数中找到损失计算部分，添加以下代码
                    # 处理MRC任务
                    mrc_indices = [i for i, t in enumerate(batch_task_types) if t == 'mrc']
                    if mrc_indices:
                        # 确保batch中包含start_position和end_position
                        if 'start_position' in batch and 'end_position' in batch:
                            # 获取MRC任务的开始和结束位置
                            start_positions = batch['start_position'].to(device)
                            end_positions = batch['end_position'].to(device)
                            
                            # 只选择MRC类型的样本
                            mrc_start_positions = start_positions[mrc_indices]
                            mrc_end_positions = end_positions[mrc_indices]
                            
                            # 获取MRC任务输入
                            mrc_input_ids = batch_input_ids[mrc_indices]
                            mrc_attention_mask = batch_attention_mask[mrc_indices] if batch_attention_mask is not None else None
                            
                            # 调用模型获取开始和结束位置的logits
                            mrc_start_logits, mrc_end_logits = model(
                                input_ids=mrc_input_ids,
                                attention_mask=mrc_attention_mask,
                                token_type_ids=token_type_ids[mrc_indices] if token_type_ids is not None else None,
                                task_type='mrc'
                            )
                            
                            # 计算MRC损失
                            mrc_start_loss = F.cross_entropy(mrc_start_logits, mrc_start_positions)
                            mrc_end_loss = F.cross_entropy(mrc_end_logits, mrc_end_positions)
                            mrc_loss = (mrc_start_loss + mrc_end_loss) / 2
                            
                            # 添加到损失字典
                            losses['mrc'] = mrc_loss * task_weights['mrc']

                    # 计算总损失
                    if losses:
                        # 根据任务数量计算加权平均损失
                        loss = sum(losses.values()) / len(losses)
                    else:
                        # 如果没有可计算的损失，使用一个小的虚拟损失
                        loss = torch.tensor(0.01, device=device, requires_grad=True)
                
                # 使用混合精度训练优化反向传播
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                
                scheduler.step()
                
                total_loss += loss.item()
                global_step += 1
            
            progress_bar.set_postfix({"Loss": f"{total_loss/(batch_idx+1):.4f}"})
            
            if global_step % eval_steps == 0:                
                val_results = evaluate(model, val_loader, device, tokenizer)
                results_history.append({
                    'step': global_step,
                    'epoch': epoch+1,
                    'val_results': val_results
                })
                
                # 在Kaggle中显示美观的验证结果
                display(HTML(f"""
                <h4>Validation Results at Step {global_step}</h4>
                <ul>
                    <li>Overall Accuracy: {val_results.get('overall_accuracy', 0):.4f}</li>
                    <li>Classify Accuracy: {val_results.get('classify_accuracy', 0):.4f}</li>
                    <li>NLI Accuracy: {val_results.get('nli_accuracy', 0):.4f}</li>
                    <li>MRC Exact Match: {val_results.get('mrc_em', 0):.4f}</li>
                </ul>
                """))
                
                model.train()
                
                val_score = sum(val_results.values()) / len(val_results) if val_results else 0
                if val_score > best_val_score:
                    best_val_score = val_score
                    torch.save(model.state_dict(), best_model_path)
                    print(f"New best model saved with score: {best_val_score:.4f} at {best_model_path}")
        
        # 每个epoch结束后的评估
        val_results = evaluate(model, val_loader, device, tokenizer)
        results_history.append({
            'epoch': epoch+1,
            'val_results': val_results
        })
        
        display(HTML(f"""
        <h3>End of Epoch {epoch+1} Validation Results</h3>
        <ul>
            <li>Overall Accuracy: {val_results.get('overall_accuracy', 0):.4f}</li>
            <li>Classify Accuracy: {val_results.get('classify_accuracy', 0):.4f}</li>
            <li>NLI Accuracy: {val_results.get('nli_accuracy', 0):.4f}</li>
            <li>MRC Exact Match: {val_results.get('mrc_em', 0):.4f}</li>
        </ul>
        """))
        
        val_score = sum(val_results.values()) / len(val_results) if val_results else 0
        if val_score > best_val_score:
            best_val_score = val_score
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with score: {best_val_score:.4f} at {best_model_path}")
            
        # GPU内存回收
        torch.cuda.empty_cache()
    
    return results_history

def evaluate_mrc_only(model, mrc_loader, device, tokenizer):
    """增强版MRC专项评估函数，提供更详细的分析"""
    model.eval()
    mrc_results = []
    all_predictions = []
    all_targets = []
    all_inputs = []
    all_contexts = []  # 存储原始上下文
    all_questions = []  # 存储问题部分
    
    with torch.no_grad():
        for batch in tqdm(mrc_loader, desc="Evaluating MRC"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch.get('attention_mask', None)
            if attention_mask is not None:
                attention_mask = attention_mask.to(device)
            token_type_ids = batch.get('token_type_ids', None)
            if token_type_ids is not None:
                token_type_ids = token_type_ids.to(device)
            target_texts = batch['target_text']
            
            # MRC前向传播
            start_logits, end_logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                task_type='mrc'
            )
            
            # 对每个样本计算预测答案
            for i in range(input_ids.size(0)):
                # 获取原始输入文本
                input_text = tokenizer.decode(input_ids[i].tolist(), skip_special_tokens=True)
                
                # 分离上下文和问题
                context, question = "", input_text
                if '[SEP]' in input_text:
                    parts = input_text.split('[SEP]', 1)
                    context = parts[0].strip()
                    question = parts[1].strip() if len(parts) > 1 else ""
                
                # 使用改进的预测方法
                predicted_answer = postprocess_mrc_prediction(
                    input_ids[i].cpu().numpy(), 
                    start_logits[i].cpu(), 
                    end_logits[i].cpu(),
                    tokenizer,
                    n_best_size=5
                )
                
                # 备用方法：如果预测为空或明显不合理，尝试另一种方法
                if not predicted_answer or len(predicted_answer) < 2:
                    start_idx = torch.argmax(start_logits[i]).item()
                    end_idx = torch.argmax(end_logits[i]).item()
                    
                    # 确保end_idx >= start_idx
                    if end_idx < start_idx:
                        end_idx = start_idx
                    
                    # 限制答案长度
                    if end_idx - start_idx > 30:
                        end_idx = start_idx + 30
                    
                    # 解码并后处理
                    predicted_answer = tokenizer.decode(
                        input_ids[i][start_idx:end_idx+1].tolist(),
                        skip_special_tokens=True,
                        clean_up_tokenization_spaces=True
                    )
                
                # 后处理清理
                predicted_answer = predicted_answer.replace(" ", "")
                predicted_answer = postprocess_mrc_answer(predicted_answer, target_texts[i])
                
                target_answer = target_texts[i].strip()
                
                # 计算匹配指标
                exact_match = predicted_answer == target_answer
                
                # 字符重叠度计算
                pred_chars = set(predicted_answer)
                target_chars = set(target_answer)
                char_overlap = len(pred_chars.intersection(target_chars)) / max(len(target_chars), 1) if target_chars else 0
                
                # 收集结果数据
                all_predictions.append(predicted_answer)
                all_targets.append(target_answer)
                all_inputs.append(input_text)
                all_contexts.append(context)
                all_questions.append(question)
                
                mrc_results.append({
                    'predicted': predicted_answer,
                    'target': target_answer,
                    'exact_match': exact_match,
                    'char_overlap': char_overlap,
                    'context': context[:100] + "..." if len(context) > 100 else context,  # 截断过长上下文
                    'question': question
                })
    
    # 计算详细指标
    metrics = calculate_mrc_metrics(all_predictions, all_targets)
    
    # 增强的错误分析
    error_analysis = analyze_mrc_errors(all_predictions, all_targets, all_inputs, all_contexts, all_questions)
    
    # 打印结果摘要
    print(f"\n【MRC评估摘要】- 样本数: {len(mrc_results)}")
    print(f"精确匹配率(EM): {metrics['exact_match']:.4f}")
    print(f"F1分数: {metrics['f1']:.4f}")
    print(f"精确率: {metrics['precision']:.4f}")
    print(f"召回率: {metrics['recall']:.4f}")
    print(f"部分匹配率: {metrics['partial_match']:.4f}")
    
    # 可视化错误类型分布
    print("\n【错误类型分布】")
    for category, stats in error_analysis['error_stats'].items():
        print(f"  {category}: {stats['count']} 例 ({stats['percentage']:.1f}%)")
    
    # 可视化答案长度分析
    print("\n【答案长度分析】")
    for length_range, count in error_analysis['length_stats'].items():
        print(f"  {length_range}: {count['count']} 例 (EM率: {count['em_rate']:.2f})")
    
    # 打印典型错误示例
    print("\n【典型错误示例】")
    for category, examples in error_analysis['error_examples'].items():
        if examples:
            print(f"\n{category} 示例:")
            for i, example in enumerate(examples[:2]):  # 每类展示2个示例
                print(f"  问题: {example.get('question', '未知')}")
                print(f"  目标: '{example['target']}'")
                print(f"  预测: '{example['prediction']}'")
                print(f"  重叠率: {example.get('char_overlap', 0):.2f}")
                print(f"  ---")
    
    # 成功案例分析
    success_cases = [r for r in mrc_results if r['exact_match']]
    print(f"\n【成功案例分析】- 共{len(success_cases)}个精确匹配")
    if success_cases:
        success_lengths = [len(case['target']) for case in success_cases]
        avg_success_len = sum(success_lengths) / len(success_lengths)
        print(f"  成功案例平均答案长度: {avg_success_len:.1f}字符")
        print("\n成功示例:")
        for case in random.sample(success_cases, min(3, len(success_cases))):
            print(f"  问题: {case['question']}")
            print(f"  答案: '{case['target']}'")
            print(f"  ---")
    
    return {
        'mrc_em': metrics['exact_match'],
        'mrc_f1': metrics['f1'],
        'mrc_precision': metrics['precision'],
        'mrc_recall': metrics['recall'],
        'mrc_partial': metrics['partial_match'],
        'error_analysis': error_analysis,
        'result_samples': mrc_results[:50]  # 保存部分结果用于后续分析
    }

## 5.2  MRC评估函数

In [23]:
def postprocess_mrc_prediction(input_ids, start_logits, end_logits, tokenizer, 
                              n_best_size=20, max_answer_length=50):
    """改进的MRC预测后处理函数"""
    # 获取所有可能的开始和结束位置
    start_indices = torch.argsort(start_logits, descending=True)[:n_best_size]
    end_indices = torch.argsort(end_logits, descending=True)[:n_best_size]
    
    valid_answers = []
    
    # 尝试不同的start-end组合
    for start_idx in start_indices:
        for end_idx in end_indices:
            # 跳过无效组合
            if end_idx < start_idx:
                continue
                
            # 跳过太长的答案
            if end_idx - start_idx + 1 > max_answer_length:
                continue
                
            # 计算这个组合的分数
            score = start_logits[start_idx] + end_logits[end_idx]
            
            # 解码这个答案
            answer = tokenizer.decode(input_ids[start_idx:end_idx+1]).strip()
            
            # 后处理逻辑
            answer = answer.replace(" ", "")  # 移除空格
            answer = answer.rstrip(",.;:?!，。；：？！")  # 移除尾部标点
            
            # 忽略太短或空答案
            if not answer or len(answer) <= 1:
                continue
                
            valid_answers.append({
                'text': answer,
                'score': score.item(),
                'start': start_idx.item(),
                'end': end_idx.item()
            })
    
    # 如果没有找到有效答案，尝试最基本的预测
    if not valid_answers:
        best_start = torch.argmax(start_logits).item()
        best_end = torch.argmax(end_logits).item()
        
        if best_end < best_start:
            best_end = best_start
            
        answer = tokenizer.decode(input_ids[best_start:best_end+1]).strip().replace(" ", "")
        
        valid_answers.append({
            'text': answer,
            'score': (start_logits[best_start] + end_logits[best_end]).item(),
            'start': best_start,
            'end': best_end
        })
    
    # 根据分数排序
    valid_answers = sorted(valid_answers, key=lambda x: x['score'], reverse=True)
    
    # 返回最佳答案
    return valid_answers[0]['text']

In [24]:
def postprocess_mrc_answer(raw_answer, context=None, target=None):
    """对MRC答案进行后处理"""
    # 1. 移除多余空格
    answer = raw_answer.replace(" ", "").strip('，。、；：""''！？.,;:\'\"!?')
    
    # 处理问句回显情况 - 新增规则
    if answer.endswith('是谁的作品') and not answer.endswith('？'):
        answer += '？'
    
    # 检查是否是问句但缺少问号
    if (answer.startswith('谁') or answer.startswith('什么') or 
        answer.startswith('为什么') or answer.startswith('怎么') or
        answer.startswith('如何') or answer.startswith('哪')) and not answer.endswith('？'):
        answer += '？'
    
    # 如果有目标参考，且目标以问号结尾而预测没有
    if target and target.endswith('？') and not answer.endswith('？'):
        answer += '？'
    
    # 2. 处理空答案情况
    if not answer and context:
        # 如果答案为空，尝试提取上下文中的关键片段
        # 可以使用一些启发式规则，如寻找问题相关词附近的名词短语
        return context[:20] if len(context) > 20 else context
    
    # 3. 截断过长答案，保持在合理范围内
    if len(answer) > 50:  # 设置最大长度阈值
        answer = answer[:50]
    
    # 4. 如果答案太短，且有原始上下文可参考，尝试扩展
    if len(answer) < 2 and context and answer in context:
        # 寻找更完整的答案片段
        idx = context.find(answer)
        if idx >= 0:
            # 尝试从上下文中提取更长的合理片段
            left_bound = max(0, idx - 5)
            right_bound = min(len(context), idx + len(answer) + 5)
            extended = context[left_bound:right_bound]
            # 如果扩展后更合理，则使用扩展后的答案
            if len(extended) > len(answer) and not (extended.startswith('，') or extended.startswith('。')):
                answer = extended
    
    return answer

def calculate_mrc_metrics(predictions, targets):
    """计算MRC的全面评估指标"""
    if not predictions or not targets:
        return {"exact_match": 0, "f1": 0, "precision": 0, "recall": 0, "partial_match": 0}
        
    exact_match = 0
    f1_scores = 0
    precision_scores = 0
    recall_scores = 0
    partial_matches = 0
    char_overlaps = 0
    
    results = {}
    
    for pred, target in zip(predictions, targets):
        pred = pred.strip()
        target = target.strip()
        
        # 精确匹配
        if pred == target:
            exact_match += 1
            f1_scores += 1
            precision_scores += 1
            recall_scores += 1
            partial_matches += 1
            char_overlaps += 1
            continue
            
        # 字符级别评估
        pred_chars = set(pred)
        target_chars = set(target)
        
        if pred_chars and target_chars:  # 确保非空
            common_chars = pred_chars.intersection(target_chars)
            
            # 字符重叠率
            char_overlap = len(common_chars) / max(len(target_chars), 1)
            char_overlaps += char_overlap
            
            # 精确率和召回率
            if pred_chars:
                precision = len(common_chars) / len(pred_chars)
                precision_scores += precision
            
            if target_chars:
                recall = len(common_chars) / len(target_chars)
                recall_scores += recall
            
            # F1分数
            if precision + recall > 0:
                f1 = (2 * precision * recall) / (precision + recall)
                f1_scores += f1
                
                # 部分匹配
                if f1 >= 0.5:
                    partial_matches += 1
        
        # 如果是空预测或空目标的特殊情况
        if not pred and target:
            # 空预测，非空目标
            precision_scores += 1.0  # 精确率为1，因为没有错误预测
            # 召回率为0，因为未找到目标
        elif pred and not target:
            # 非空预测，空目标
            recall_scores += 1.0  # 召回率为1，因为找到了所有目标(没有目标)
            # 精确率为0，因为所有预测都是错误的
    
    num_samples = len(predictions)
    
    results = {
        "exact_match": exact_match / num_samples,
        "f1": f1_scores / num_samples,
        "precision": precision_scores / num_samples,
        "recall": recall_scores / num_samples,
        "partial_match": partial_matches / num_samples,
        "char_overlap": char_overlaps / num_samples
    }
    
    return results

In [25]:
def longest_common_substring(s1, s2):
    m = len(s1)
    n = len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    max_len = 0
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
                max_len = max(max_len, dp[i][j])
    
    return max_len

def analyze_mrc_errors(predictions, targets, input_texts=None, contexts=None, questions=None, max_examples=10):
    """增强版MRC错误分析函数"""
    if not predictions or len(predictions) != len(targets):
        return {"error_stats": {}, "error_examples": {}, "length_stats": {}}
    
    error_types = {
        "长度差异": [],      # 长度差异较大
        "部分匹配": [],      # 部分匹配但不完全
        "完全不匹配": [],    # 完全不匹配
        "标点符号差异": [],  # 只有标点符号差异
        "字符错位": [],      # 有相同字符但顺序错误
        "答案边界错误": []   # 答案边界不准确(太长或太短)
    }
    
    # 长度分析
    length_stats = {
        "1-5字符": {"count": 0, "correct": 0, "em_rate": 0},
        "6-10字符": {"count": 0, "correct": 0, "em_rate": 0},
        "11-20字符": {"count": 0, "correct": 0, "em_rate": 0},
        "21-50字符": {"count": 0, "correct": 0, "em_rate": 0},
        "50字符以上": {"count": 0, "correct": 0, "em_rate": 0}
    }
    
    for i, (pred, target) in enumerate(zip(predictions, targets)):
        # 长度统计
        target_len = len(target)
        if 1 <= target_len <= 5:
            length_range = "1-5字符"
        elif 6 <= target_len <= 10:
            length_range = "6-10字符"
        elif 11 <= target_len <= 20:
            length_range = "11-20字符"
        elif 21 <= target_len <= 50:
            length_range = "21-50字符"
        else:
            length_range = "50字符以上"
            
        length_stats[length_range]["count"] += 1
        if pred == target:
            length_stats[length_range]["correct"] += 1
            continue  # 跳过正确预测
            
        # 错误分析
        # 长度差异分析
        len_diff = abs(len(pred) - len(target))
        len_ratio = max(len(pred), len(target)) / max(min(len(pred), len(target)), 1)
        
        # 字符重叠分析
        pred_chars = set(pred)
        target_chars = set(target)
        common_chars = pred_chars.intersection(target_chars)
        char_overlap = len(common_chars) / max(len(target_chars), 1) if target_chars else 0
        
        # 字符序列分析
        longest_common_substr_len = longest_common_substring(pred, target)
        sequence_match_ratio = longest_common_substr_len / max(len(target), 1)
        
        # 标点符号差异
        punctuation = set("，。！？""''「」『』（）：；、.,?!\"':;()[]{}")
        pred_no_punct = ''.join([c for c in pred if c not in punctuation])
        target_no_punct = ''.join([c for c in target if c not in punctuation])
        
        # 错误类型分类
        if pred_no_punct == target_no_punct:
            category = "标点符号差异"
        elif len_ratio > 2 or len_diff > 10:
            # 检查是否答案边界不准确
            if target in pred or (len(target) > 5 and any(target[i:i+5] in pred for i in range(len(target)-4))):
                category = "答案边界错误"
            else:
                category = "长度差异"
        elif char_overlap >= 0.7 and sequence_match_ratio < 0.5:
            category = "字符错位"
        elif char_overlap >= 0.3:
            category = "部分匹配"
        else:
            category = "完全不匹配"
        
        # 构建错误示例
        example = {
            "id": i,
            "prediction": pred,
            "target": target,
            "char_overlap": char_overlap,
            "len_diff": len_diff
        }
        
        # 添加额外上下文信息
        if input_texts and i < len(input_texts):
            example["input_text"] = input_texts[i]
        if contexts and i < len(contexts):
            example["context"] = contexts[i]
        if questions and i < len(questions):
            example["question"] = questions[i]
        
        # 添加到对应类别
        if len(error_types[category]) < max_examples:
            error_types[category].append(example)
    
    # 计算长度区间的正确率
    for length_range in length_stats:
        count = length_stats[length_range]["count"]
        correct = length_stats[length_range]["correct"]
        em_rate = correct / count if count > 0 else 0
        length_stats[length_range]["em_rate"] = em_rate
    
    # 统计各类错误的比例
    total_errors = sum(len(errors) for errors in error_types.values())
    error_stats = {
        category: {
            "count": len(errors),
            "percentage": len(errors) / max(total_errors, 1) * 100
        }
        for category, errors in error_types.items()
    }
    
    return {
        "error_stats": error_stats,
        "error_examples": error_types,
        "length_stats": length_stats
    }

In [26]:
def train_multitask(model, train_loaders, val_loaders, optimizer, scheduler, device, 
                   best_model_path, tokenizer, num_epochs=12, eval_steps=150):
    """按任务类型分别训练模型 - 优化版本"""
    best_val_score = 0
    global_step = 0
    scaler = torch.cuda.amp.GradScaler()
    patience = 5  # 增加耐心值
    early_stopping_counter = 0
    min_improvement = 0.005  # 最小改进阈值
    
    # 记录结果历史
    results_history = []
    
    # 每个任务的性能指标记录
    task_performance = {
        'classify': 0.0,
        'nli': 0.0,
        'mrc': 0.0
    }
    
    # 任务权重 - 更高的初始值
    task_weights = {
        'classify': 5.0,  # 降低初始权重，更稳定
        'nli': 5.0, 
        'mrc': 10.0
    }
    
    # 任务采样概率 - 更均衡
    task_sampling_weights = {
        'classify': 3.0,
        'nli': 3.0,
        'mrc': 5.0  # 更有利于MRC训练
    }
    
    task_lr_multipliers = {
        'classify': 10.0,
        'nli': 20.0,
        'mrc': 250.0  # 提高MRC头部的学习率
    }
    
    # 设置学习率下限，防止学习率过低
    min_lr_multiplier = {
        'classify': 5.0,  # 确保分类任务有足够但不过高的学习率
        'nli': 5.0,
        'mrc': 50.0
    }
    
    # 使用固定的学习率和权重几个epoch，让模型先稳定
    stabilization_epochs = 2
    
    # 使用动态累积梯度步数
    grad_accum_steps = {'classify': 4, 'nli': 2, 'mrc': 1}  # MRC更新更频繁
    current_grad_steps = {'classify': 0, 'nli': 0, 'mrc': 0}
    
    # 添加学习率预热和周期
    warmup_steps = 100
    current_warmup = 0
    
    # 添加梯度裁剪阈值
    max_grad_norm = 1.0
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        task_losses = {'classify': 0.0, 'nli': 0.0, 'mrc': 0.0}
        task_counts = {'classify': 0, 'nli': 0, 'mrc': 0}
        batch_count = 0
        
        # 为每个任务创建迭代器
        task_iterators = {task: iter(loader) for task, loader in train_loaders.items() if len(loader) > 0}
        active_tasks = list(task_iterators.keys())
        
        # 估计总批次数
        total_batches = sum(len(loader) for loader in train_loaders.values())
        progress_bar = tqdm(range(total_batches), desc=f"Epoch {epoch+1}/{num_epochs}")
        
        # 按序轮流训练各任务
        while active_tasks:
            # 确保模型处于训练模式
            if not model.training:
                model.train()
                
            # 使用当前权重进行任务采样
            weights = [task_sampling_weights[t] for t in active_tasks]
            total_weight = sum(weights)
            normalized_weights = [w/total_weight for w in weights]
            
            # 根据权重选择任务类型
            task_type = random.choices(active_tasks, weights=normalized_weights, k=1)[0]
                
            try:
                batch = next(task_iterators[task_type])
                current_grad_steps[task_type] += 1
                
                # 处理批次
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch.get('attention_mask', None)
                if attention_mask is not None:
                    attention_mask = attention_mask.to(device)
                
                token_type_ids = batch.get('token_type_ids', None)
                if token_type_ids is not None:
                    token_type_ids = token_type_ids.to(device)
                
                # 每个累积周期的第一步清零梯度
                if current_grad_steps[task_type] == 1:
                    optimizer.zero_grad()
                
                # 使用混合精度训练
                with torch.cuda.amp.autocast():
                    batch_processed = False
                    
                    if task_type in ['classify', 'nli']:
                        # 获取标签
                        labels = batch['label'].to(device)
                        
                        # 前向传播
                        outputs = model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            task_type=task_type
                        )
                        
                        # 计算损失
                        if task_type == 'nli':
                            loss_fn = LabelSmoothingCrossEntropy(smoothing=0.1)
                            loss = loss_fn(outputs, labels) * task_weights[task_type]
                        elif task_type == 'classify':
                            # 计算类别权重 - 可以根据batch中出现的各类别频率计算
                            if 'target_text' in batch:
                                targets = batch['target_text']
                                target_counter = Counter(targets)
                                total = len(targets)
                                # 计算反频率权重: 频率越低权重越高
                                class_weights = {label: total / (count * len(target_counter)) for label, count in target_counter.items()}
                                # 将字典转换为列表，并根据answer_choices的顺序
                                if 'answer_choices' in batch:
                                    choices = batch['answer_choices'][0] if isinstance(batch['answer_choices'][0], list) else batch['answer_choices']
                                    weight_list = [class_weights.get(choice, 1.0) for choice in choices]
                                    # 应用类别权重的交叉熵损失
                                    loss = weighted_cross_entropy_loss(outputs, labels, weight_list) * task_weights[task_type]
                                else:
                                    # 回退到Focal Loss
                                    loss = focal_loss(outputs, labels, gamma=3.0) * task_weights[task_type]  # 增加gamma值
                            else:
                                # 如果没有标签文本，使用增强版Focal Loss
                                loss = focal_loss(outputs, labels, gamma=3.0) * task_weights[task_type]  # 增加gamma值
                        else:
                            loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
                            loss = loss_fn(outputs, labels) * task_weights[task_type]
                        
                        # 反向传播
                        scaler.scale(loss).backward()
                        batch_processed = True
                    
                    elif task_type == 'mrc':
                        # 获取必要输入
                        start_positions = batch['start_position'].to(device)
                        end_positions = batch['end_position'].to(device)
                        
                        # 额外的输入用于增强损失计算
                        target_texts = batch['target_text'] if 'target_text' in batch else None
                        
                        # 检查批次大小，如果太大则分批处理
                        batch_size = input_ids.shape[0]
                        if batch_size > 8:  # 如果批次太大，分批处理
                            sub_batch_size = 8
                            total_sub_loss = 0.0
                            
                            for i in range(0, batch_size, sub_batch_size):
                                end_idx = min(i + sub_batch_size, batch_size)
                                sub_input_ids = input_ids[i:end_idx]
                                sub_attention_mask = attention_mask[i:end_idx] if attention_mask is not None else None
                                sub_token_type_ids = token_type_ids[i:end_idx] if token_type_ids is not None else None
                                sub_start_positions = start_positions[i:end_idx]
                                sub_end_positions = end_positions[i:end_idx]
                                
                                # 前向传播
                                with torch.cuda.amp.autocast():
                                    sub_start_logits, sub_end_logits = model(
                                        input_ids=sub_input_ids,
                                        attention_mask=sub_attention_mask,
                                        token_type_ids=sub_token_type_ids,
                                        task_type='mrc'
                                    )
                                
                                # 计算损失
                                sub_loss = compute_mrc_loss(
                                    sub_start_logits, sub_end_logits,
                                    sub_start_positions, sub_end_positions,
                                    answer_texts=target_texts[i:end_idx] if target_texts else None,
                                    input_ids=sub_input_ids,
                                    tokenizer=tokenizer
                                ) * task_weights[task_type] / (batch_size / sub_batch_size)
                                
                                # 累计损失
                                total_sub_loss += sub_loss
                            
                            # 对累计损失执行反向传播
                            scaler.scale(total_sub_loss).backward()
                            loss = total_sub_loss  # 使用累计损失作为整体损失
                            batch_processed = True
                            
                        else:
                            # 批次不大，直接处理
                            with torch.cuda.amp.autocast():
                                start_logits, end_logits = model(
                                    input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    token_type_ids=token_type_ids,
                                    task_type='mrc'
                                )
                            
                            # 使用增强的MRC损失
                            loss = compute_mrc_loss(
                                start_logits, end_logits, 
                                start_positions, end_positions,
                                answer_texts=target_texts,
                                input_ids=input_ids,
                                tokenizer=tokenizer
                            ) * task_weights[task_type]
                            
                            # 反向传播
                            scaler.scale(loss).backward()
                            batch_processed = True
                            
                # 当达到累积步数时更新参数
                if current_grad_steps[task_type] >= grad_accum_steps[task_type]:
                    # 梯度裁剪
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                    
                    # 优化器步进
                    scaler.step(optimizer)
                    scaler.update()
                    
                    # 学习率调度
                    if current_warmup < warmup_steps:
                        # 预热阶段
                        lr_scale = min(1.0, current_warmup / warmup_steps)
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr_scale * param_group['initial_lr']
                        current_warmup += 1
                    else:
                        scheduler.step()
                    
                    # 重置累积步数
                    current_grad_steps[task_type] = 0
                
                # 更新计数和损失
                task_losses[task_type] += loss.item()
                task_counts[task_type] += 1
                total_loss += loss.item()
                batch_count += 1
                global_step += 1
                
                # 更新进度条
                progress_bar.update(1)
                avg_loss = total_loss / batch_count if batch_count > 0 else 0
                progress_bar.set_postfix({
                    "Loss": f"{avg_loss:.4f}", 
                    "C": f"{task_losses['classify']/max(task_counts['classify'],1):.2f}",
                    "N": f"{task_losses['nli']/max(task_counts['nli'],1):.2f}", 
                    "M": f"{task_losses['mrc']/max(task_counts['mrc'],1):.2f}"
                })
                
                # 周期性评估
                if global_step % eval_steps == 0:
                    # 保存当前训练模式
                    was_training = model.training
                    
                    # 设置为评估模式
                    model.eval()
                    
                    try:
                        # 执行评估
                        val_results = evaluate_multitask(model, val_loaders, device, tokenizer)
                        results_history.append({
                            'step': global_step,
                            'epoch': epoch+1,
                            'val_results': val_results
                        })
                    
                        # 更新任务性能指标
                        for task in task_performance.keys():
                            if f'{task}_accuracy' in val_results:
                                task_performance[task] = val_results[f'{task}_accuracy']
                            elif task == 'mrc' and 'mrc_em' in val_results:
                                task_performance[task] = val_results['mrc_em']
                        
                        # 动态调整任务权重和采样权重
                        task_weights, task_sampling_weights = _adjust_task_weights(task_performance, task_weights, task_sampling_weights, val_results)
                        
                        # 显示结果
                        display(HTML(f"""
                        <h4>Validation Results at Step {global_step}</h4>
                        <ul>
                            <li>Overall Accuracy: {val_results.get('overall_accuracy', 0):.4f}</li>
                            <li>Classify Accuracy: {val_results.get('classify_accuracy', 0):.4f}</li>
                            <li>NLI Accuracy: {val_results.get('nli_accuracy', 0):.4f}</li>
                            <li>MRC Exact Match: {val_results.get('mrc_em', 0):.4f}</li>
                            <li>MRC F1: {val_results.get('mrc_f1', 0):.4f}</li>
                        </ul>
                        <p>Current Task Weights: C={task_weights['classify']:.1f}, N={task_weights['nli']:.1f}, M={task_weights['mrc']:.1f}</p>
                        <p>Current Sampling Weights: C={task_sampling_weights['classify']:.1f}, N={task_sampling_weights['nli']:.1f}, M={task_sampling_weights['mrc']:.1f}</p>
                        """))
                    
                    finally:
                        # 无论评估是否成功，都确保恢复训练模式
                        if was_training:
                            model.train()
                    
                    # 保存最佳模型
                    val_score = sum(val_results.values()) / len(val_results) if val_results else 0
                    if val_score > best_val_score:
                        best_val_score = val_score
                        torch.save(model.state_dict(), best_model_path)
                        print(f"New best model saved with score: {best_val_score:.4f}")
                        early_stopping_counter = 0
                    else:
                        early_stopping_counter += 1
                        
                    # MRC专项评估
                    if 'mrc' in val_loaders and global_step % (2 * eval_steps) == 0:
                        # 保存当前模式
                        current_mode = model.training
                        model.eval()  # 设置为评估模式
                        try:
                            mrc_val_results = evaluate_mrc_only(model, val_loaders['mrc'], device, tokenizer)
                        finally:
                            # 恢复原来的模式
                            if current_mode:
                                model.train()
                            
            except StopIteration:
                # 当前任务的批次已用完
                active_tasks.remove(task_type)
        
        # 每个epoch结束后的评估
        model.eval()  # 确保在评估状态
        try:
            val_results = evaluate_multitask(model, val_loaders, device, tokenizer)
            results_history.append({
                'epoch': epoch+1,
                'val_results': val_results
            })
            
            # 显示每个任务的平均损失
            task_avg_losses = {t: task_losses[t]/max(task_counts[t], 1) for t in task_losses}
            
            display(HTML(f"""
            <h3>End of Epoch {epoch+1} Validation Results</h3>
            <ul>
                <li>Overall Accuracy: {val_results.get('overall_accuracy', 0):.4f}</li>
                <li>Classify Accuracy: {val_results.get('classify_accuracy', 0):.4f} (Loss: {task_avg_losses['classify']:.4f})</li>
                <li>NLI Accuracy: {val_results.get('nli_accuracy', 0):.4f} (Loss: {task_avg_losses['nli']:.4f})</li>
                <li>MRC Exact Match: {val_results.get('mrc_em', 0):.4f} (Loss: {task_avg_losses['mrc']:.4f})</li>
                <li>MRC F1: {val_results.get('mrc_f1', 0):.4f}</li>
            </ul>
            """))
            
            # 更新任务性能并动态调整权重
            for task in task_performance.keys():
                if f'{task}_accuracy' in val_results:
                    task_performance[task] = val_results[f'{task}_accuracy']
                elif task == 'mrc' and 'mrc_em' in val_results:
                    task_performance[task] = val_results['mrc_em']
            
            # 调整梯度累积步数，为表现较弱的任务提供更多更新机会
            min_performance = min(task_performance.values())
            for task, perf in task_performance.items():
                if perf == min_performance:
                    grad_accum_steps[task] = max(1, grad_accum_steps[task] - 1)  # 减少梯度累积步数
                elif perf > 0.8:  # 表现很好的任务可以增加梯度累积步数
                    grad_accum_steps[task] = min(4, grad_accum_steps[task] + 1)
            
            print(f"调整梯度累积步数: {grad_accum_steps}")
            
            val_score = sum(val_results.values()) / len(val_results) if val_results else 0
            if val_score > best_val_score + min_improvement:  # 添加最小改进阈值
                best_val_score = val_score
                torch.save(model.state_dict(), best_model_path)
                print(f"New best model saved with score: {best_val_score:.4f}")
                early_stopping_counter = 0  # 重置计数器
            elif val_score > best_val_score:
                # 有改进但不显著，仍然保存模型但不重置计数器
                best_val_score = val_score
                torch.save(model.state_dict(), best_model_path)
                print(f"Minor improvement, model saved with score: {best_val_score:.4f}")
                early_stopping_counter += 0.5  # 只增加半个计数
            else:
                early_stopping_counter += 1
        finally:
            # 恢复训练模式，为下一轮做准备
            model.train()
        
        # 每个epoch结束后对MRC进行专门评估
        if 'mrc' in val_loaders:
            model.eval()
            try:
                mrc_metrics = evaluate_mrc_only(model, val_loaders['mrc'], device, tokenizer)
                
                # 输出详细MRC指标
                display(HTML(f"""
                <h4>MRC专项评估</h4>
                <ul>
                    <li>精确匹配率(EM): {mrc_metrics['mrc_em']:.4f}</li>
                    <li>F1分数: {mrc_metrics['mrc_f1']:.4f}</li>
                    <li>精确率: {mrc_metrics['mrc_precision']:.4f}</li>
                    <li>召回率: {mrc_metrics['mrc_recall']:.4f}</li>
                </ul>
                """))
                
                # 只在稳定期后才动态调整权重
                if epoch >= stabilization_epochs:
                    # 更新任务性能并动态调整权重，但限制adjust幅度
                    task_weights, task_sampling_weights = _adjust_task_weights(
                        task_performance, task_weights, task_sampling_weights, 
                        val_results, max_weight_change=1.1  # 限制最大变化幅度
                    )
                # 如果在稳定期内，打印信息
                else:
                    print(f"在稳定期内(第{epoch+1}/{stabilization_epochs}个epoch)，保持权重不变")
                    print(f"当前任务权重: {task_weights}")
                    print(f"当前采样权重: {task_sampling_weights}")
            finally:
                # 恢复训练模式
                model.train()
        
        # 早停检查
        if early_stopping_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break
            
        # GPU内存回收
        torch.cuda.empty_cache()
    
    return results_history

In [27]:
# 新增的权重动态调整方法
def _adjust_task_weights(task_performance, task_weights, task_sampling_weights, 
                        val_results, max_weight_change=1.5):  # 增加最大变化幅度
    """动态调整任务权重和采样权重 - 更激进版本"""
    # 找出性能最差的任务
    min_perf_task = min(task_performance.items(), key=lambda x: x[1])[0]
    
    # 找出性能最好的任务
    max_perf_task = max(task_performance.items(), key=lambda x: x[1])[0]
    
    # classify任务特殊处理 - 新增
    classify_balanced_acc = val_results.get('classify_balanced_accuracy', 0)
    classify_rare_acc = val_results.get('classify_rare_accuracy', 0)
    classify_common_acc = val_results.get('classify_common_accuracy', 0)
    
    # MRC任务特殊处理
    mrc_em = val_results.get('mrc_em', 0)
    mrc_f1 = val_results.get('mrc_f1', 0)
    
    # 保存原始权重以便限制变化幅度
    old_weights = {task: weight for task, weight in task_weights.items()}
    old_sampling_weights = {task: weight for task, weight in task_sampling_weights.items()}
    
    # 权重调整策略
    for task in task_weights:
        # 基于当前性能调整权重
        if task == min_perf_task:
            # 更激进地增加权重和采样概率
            task_weights[task] = min(
                task_weights[task] * min(max_weight_change, 1.5),  # 最多增加50%
                task_weights[task] * 1.5
            )
            task_sampling_weights[task] = min(
                task_sampling_weights[task] * min(max_weight_change, 1.8),  # 最多增加80%
                task_sampling_weights[task] * 1.8
            )
        elif task == max_perf_task:
            # 减少权重和采样概率，但限制幅度
            task_weights[task] = max(
                task_weights[task] / min(max_weight_change, 1.2),  # 最多减少20%
                task_weights[task] * 0.8
            )
            task_sampling_weights[task] = max(
                task_sampling_weights[task] / min(max_weight_change, 1.2),
                task_sampling_weights[task] * 0.8
            )
    
    # MRC特殊处理 - 降低MRC权重，让其他任务有更多学习空间
    if 'mrc' in task_weights and task_weights['mrc'] > 15.0:
        # 适当降低MRC的权重
        task_weights['mrc'] = max(15.0, task_weights['mrc'] * 0.9)  # 设定最低值为15
        
    # 分类任务特别处理 - 大幅增加权重
    if 'classify' in task_weights:
        classify_acc = task_performance.get('classify', 0)
        
        # 根据分类性能更激进调整权重
        if classify_acc < 0.5:  # 分类任务表现较差
            # 显著增加权重和采样频率
            task_weights['classify'] = min(task_weights['classify'] * 2.0, 20.0)  # 原来是1.3,8.0
            task_sampling_weights['classify'] = min(task_sampling_weights['classify'] * 2.0, 8.0)  # 原来是1.4,5.0
            print(f"分类任务表现较差 (Accuracy: {classify_acc:.4f})，显著增加其权重")
        elif classify_acc < 0.7:  # 分类任务表现一般
            # 适度增加权重
            task_weights['classify'] = min(task_weights['classify'] * 1.5, 15.0)  # 原来是1.1,6.0
            task_sampling_weights['classify'] = min(task_sampling_weights['classify'] * 1.5, 6.0)  # 原来是1.2,4.0
            print(f"分类任务表现一般 (Accuracy: {classify_acc:.4f})，适度增加其权重")
        elif classify_acc > 0.8:  # 分类任务表现很好
            # 稍微降低权重，让其他任务有更多学习机会
            task_weights['classify'] = max(task_weights['classify'] / 1.05, 5.0)  # 原来是1.05,2.0
            task_sampling_weights['classify'] = max(task_sampling_weights['classify'] / 1.05, 3.0)  # 原来是1.05,1.5
            print(f"分类任务表现很好 (Accuracy: {classify_acc:.4f})，略微降低其权重")
    
    print(f"调整后的任务权重: {task_weights}")
    print(f"调整后的采样权重: {task_sampling_weights}")
    
    return task_weights, task_sampling_weights

# 6 主函数

In [28]:
def main():
    # 参数设置
    # 检测运行环境并设置相应的路径
    is_kaggle = os.path.exists('/kaggle') and '/kaggle/working' in os.getcwd()

    # 确保使用正确的项目根目录
    project_root = os.path.dirname(os.path.abspath('__file__'))
    possible_paths = [
        os.path.dirname(os.path.abspath('__file__')),  # 当前文件所在目录
        os.getcwd(),  # 当前工作目录
        '/root/Code/Multi-task Project'  # 从错误消息推断的完整路径
    ]

    for path in possible_paths:
        if os.path.exists(path):
            project_root = path
            break

    print(f"Using project root: {project_root}")

    # 定义best_model_path变量
    if is_kaggle:
        model_path = '/kaggle/working/best_multitask_transformer.pt'
        best_model_path = model_path
        output_dir = '/kaggle/working/outputs'
        data_file = '/kaggle/input/multitask-data/data.jsonl'
    else:
        model_paths = [
            os.path.join(project_root, "outputs", "transformer_model", "best_multitask_transformer.pt"),
            os.path.join(project_root, "test_outputs", "multitask_model.pth"),
            os.path.join(project_root, "best_multitask_transformer.pt"),
            "./best_multitask_transformer.pt"
        ]
        
        model_path = None
        for path in model_paths:
            if os.path.exists(path):
                model_path = path
                break
        
        if model_path is None:
            model_path = os.path.join(project_root, "outputs", "transformer_model", "best_multitask_transformer.pt")
            print(f"警告: 没有找到现有模型，将创建一个新模型实例")
        
        best_model_path = model_path
        output_dir = os.path.join(project_root, "outputs", "transformer_model")
        data_file = os.path.join(project_root, "data/processed", "pCLUE_train.json") 

    print(f"使用模型路径: {model_path}")
    
    # 模型训练参数
    sample_percentage = 60
    max_samples = 30000
    batch_size = 8
    epochs = 12
    lr = 2e-5
    weight_decay = 0.02
    eval_steps = 150
    seed = 42
    task_types = ["classify", "nli", "mrc"]
    
    # 修改任务初始权重 - 更高的classify权重
    task_weights = {
        'classify': 10.0,  # 显著提高分类权重
        'nli': 5.0,        # 也适当提高NLI权重
        'mrc': 15.0        # 降低MRC权重
    }
    
    # 修改任务采样概率 - 大幅增加分类任务采样概率
    task_sampling_weights = {
        'classify': 5.0,  # 大幅增加分类任务采样概率
        'nli': 2.0,       # 适当增加NLI任务采样概率
        'mrc': 3.0        # 降低MRC任务采样比例
    }
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 检查数据文件是否存在
    if not os.path.exists(data_file):
        raise FileNotFoundError(f"数据文件 '{data_file}' 不存在，请检查路径")
    
    set_seed(seed)
    device = get_device()
    
    # 显示开始信息
    display(HTML("<h2>Starting Multitask Training with Task-Specific Data Loading</h2>"))
    
    # 加载数据 - 使用并行加载版本
    all_data = parallel_load_data(
        data_file,
        sample_percentage=sample_percentage,
        max_samples=max_samples,
        task_types=task_types,
        n_workers=min(16, os.cpu_count())  # 设置适当的工作进程数
    )
    
    # 数据预处理和平衡
    print("原始数据样本数:", len(all_data))

    # 预处理：确保每个数据项都有必要的字段
    valid_data = []
    for item in all_data:
        if 'type' not in item or 'input' not in item or 'target' not in item:
            continue
        valid_data.append(item)

    if len(valid_data) != len(all_data):
        print(f"警告: 过滤了 {len(all_data) - len(valid_data)} 条无效数据项")
        all_data = valid_data

    # 处理classify任务数据 - 使用增强版平衡函数
    classify_data = [item for item in all_data if item['type'] == 'classify']
    other_data = [item for item in all_data if item['type'] != 'classify']
    
    # 应用增强的标签平衡
    if classify_data:
        try:
            print("正在对分类数据进行增强平衡...")
            balanced_classify_data = balance_classify_data_improved(classify_data, max_per_class=3000, min_per_class=200)
            # 将平衡后的classify数据与其他任务数据合并
            all_data = balanced_classify_data + other_data
            print(f"处理后数据样本数: {len(all_data)}")
        except Exception as e:
            print(f"平衡分类数据时出错: {e}")
            print("继续使用原始分类数据")
    
    # 确保每个样本都有答案选项
    for item in all_data:
        if 'answer_choices' not in item or not item['answer_choices']:
            item['answer_choices'] = ["是的", "不是"] if item['type'] == 'nli' else ["选项A", "选项B"]
        # 修复选项数量过多的问题
        elif item['type'] == 'classify' and len(item['answer_choices']) > 50:
            # 保留目标选项和随机选取其他选项，总共不超过30个
            target = item['target']
            others = [c for c in item['answer_choices'] if c != target]
            if len(others) > 29:  # 最多保留29个其他选项
                others = random.sample(others, 29)
            item['answer_choices'] = [target] + others
            random.shuffle(item['answer_choices'])  # 打乱顺序
    
    # 分割数据集
    train_size = int(0.8 * len(all_data))
    random.shuffle(all_data)
    train_data = all_data[:train_size]
    val_data = all_data[train_size:]
    
    # 加载tokenizer
    local_model_path = "./chinese-roberta-wwm-ext"
    if os.path.exists(os.path.join(local_model_path, "tokenizer_config.json")):
        tokenizer = AutoTokenizer.from_pretrained(local_model_path, local_files_only=True)
    else:
        alternative_paths = [
            os.path.join(project_root, "chinese-roberta-wwm-ext"),
            "/root/Code/Multi-task Project/chinese-roberta-wwm-ext"
        ]
        found_model = False
        for path in alternative_paths:
            if os.path.exists(os.path.join(path, "tokenizer_config.json")):
                tokenizer = AutoTokenizer.from_pretrained(path, local_files_only=True)
                found_model = True
                break
        
        if not found_model:
            raise ValueError("无法找到本地模型文件，请确认bert-base-chinese-local目录路径")
    
    # 创建任务特定的数据加载器
    train_loaders, val_loaders, label_maps = create_task_specific_dataloaders(
        train_data, val_data, tokenizer, batch_size
    )
    
    # 计算词汇表大小和标签数量
    vocab_size = tokenizer.vocab_size
    num_labels = max(len(label_map) for label_map in label_maps.values()) if label_maps else 2
    print(f"Vocabulary size: {vocab_size}")
    print(f"Max labels for classification: {num_labels}")
    
    # 创建模型
    model = MultitaskBertModel(num_labels=num_labels)
    model.to(device)
    
    # 优化器和学习率调度器设置
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.weight']
    
    task_head_params = {
        'classify': list(model.classify_head.parameters()),
        'nli': list(model.nli_head.parameters()),
        'mrc': list(model.mrc_head.parameters())
    }
    
    # 优化学习率
    optimizer_grouped_parameters = [
        # BERT层参数
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and 'bert' in n],
         'weight_decay': weight_decay, 'lr': lr, 'initial_lr': lr},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay) and 'bert' in n],
         'weight_decay': 0.0, 'lr': lr, 'initial_lr': lr},
         
        # 分类头参数 - 显著增加学习率
        {'params': task_head_params['classify'],
         'weight_decay': weight_decay, 'lr': lr * 20, 'initial_lr': lr * 20},  # 原来是lr * 5
         
        # NLI头参数 - 显著增加学习率
        {'params': task_head_params['nli'],
         'weight_decay': weight_decay, 'lr': lr * 20, 'initial_lr': lr * 20},  # 原来是lr * 10
         
        # MRC头参数 - 保持较高学习率
        {'params': task_head_params['mrc'],
         'weight_decay': weight_decay, 'lr': lr * 15, 'initial_lr': lr * 15}  # 稍微降低，原来是lr * 20
    ]
    
    # 使用AdamW替代Adam
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters)
    
    # 计算总步数
    total_steps = sum(len(loader) for loader in train_loaders.values()) * epochs
    warmup_steps = total_steps // 5  # 调整预热步骤为20%
    
    # 学习率调度策略
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    
    # 训练模型
    results_history = train_multitask(
        model,
        train_loaders,
        val_loaders,
        optimizer,
        scheduler,
        device,
        best_model_path,
        tokenizer,
        num_epochs=epochs,
        eval_steps=eval_steps
    )
    
    # 最终评估
    if os.path.exists(best_model_path):
        print(f"加载最佳模型用于最终评估: {best_model_path}")
        model.load_state_dict(torch.load(best_model_path))
        model.to(device)
    else:
        print(f"警告：无法找到最佳模型文件，使用当前模型进行评估")
    
    final_results = evaluate_multitask(model, val_loaders, device, tokenizer)
    
    display(HTML("<h2>Final Evaluation Results</h2>"))
    
    # 保存最终配置和结果
    # 提取每个任务的标签映射
    label_maps_inv = {
        task: {v: k for k, v in task_map.items()} 
        for task, task_map in label_maps.items()
    }
    with open(os.path.join(output_dir, "label_maps.json"), "w", encoding="utf-8") as f:
        json.dump(label_maps_inv, f, ensure_ascii=False, indent=2)
    print("Task-specific label maps saved for inference")
    
    # 保存模型配置信息
    config_info = {
        "vocab_size": vocab_size,
        "num_labels": num_labels,
        "task_types": task_types,
        "training_params": {
            "batch_size": batch_size,
            "epochs": epochs,
            "learning_rate": lr
        },
        "best_results": final_results
    }
    
    with open(os.path.join(output_dir, "model_config.json"), "w", encoding="utf-8") as f:
        json.dump(config_info, f, ensure_ascii=False, indent=2)
    print("Model configuration saved")

if __name__ == "__main__":
    main()

Using project root: /root/Code/Multi-task Project
使用模型路径: /root/Code/Multi-task Project/test_outputs/multitask_model.pth
使用GPU: NVIDIA GeForce RTX 4090
GPU内存总量: 23.65 GB
可用GPU数量: 1


Reading file and sampling 60% of data using 16 workers...
Total lines read: 52026
Sampled data size: 30000
原始数据样本数: 30000
正在对分类数据进行增强平衡...
分类数据共有 133 个不同的类别
类别分布情况:
  有 10 个类别的样本数为 1
  有 10 个类别的样本数为 2
  有 13 个类别的样本数为 3
  有 16 个类别的样本数为 4
  有 10 个类别的样本数为 5
  有 5 个类别的样本数为 6
  有 3 个类别的样本数为 8
  有 4 个类别的样本数为 9
  有 5 个类别的样本数为 11
  有 4 个类别的样本数为 14
已处理 100/133 个类别 (小类: 100, 中类: 0, 大类: 0)
已处理 133/133 个类别 (小类: 115, 中类: 18, 大类: 0)
原始分类数据: 13868条 -> 平衡后: 19804条
平衡后的类别分布情况:
  有 10 个类别的样本数为 12
  有 13 个类别的样本数为 18
  有 16 个类别的样本数为 24
  有 10 个类别的样本数为 30
  有 12 个类别的样本数为 200
处理后数据样本数: 35936
分类数据共有 133 个不同的类别
类别分布情况:
  有 4 个类别的样本数为 4
  有 5 个类别的样本数为 5
  有 3 个类别的样本数为 12
  有 5 个类别的样本数为 14
  有 4 个类别的样本数为 15
  有 4 个类别的样本数为 19
  有 4 个类别的样本数为 21
  有 4 个类别的样本数为 23
  有 3 个类别的样本数为 118
  有 3 个类别的样本数为 149
已处理 100/133 个类别 (小类: 100, 中类: 0, 大类: 0)
已处理 133/133 个类别 (小类: 118, 中类: 15, 大类: 0)
原始分类数据: 15794条 -> 平衡后: 30777条
平衡后的类别分布情况:
  有 5 个类别的样本数为 30
  有 5 个类别的样本数为 84
  有 4 个类别的样本数为 126
  有 4 个类别的样本数为 138
  有 43 个类别的样本数为 300


/root/miniconda3/envs/multi/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
/tmp/ipykernel_4887/4294897518.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/12:   0%|          | 0/3595 [00:00<?, ?it/s]/tmp/ipykernel_4887/4294897518.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/12:   0%|          | 4/3595 [00:01<17:38,  3.39it/s, Loss=24.8428, C=29.09, N=23.43, M=0.00]/tmp/ipykernel_4887/4294897518.py:207: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cu


【MRC诊断信息】
样本数: 1122
精确匹配率: 0.7094
平均F1分数: 0.8494
部分匹配率: 0.8503

【预测样例】
目标: '摩托车速度非常快'
预测: '摩托车速度非常快'
匹配: True

目标: '杳无音信'
预测: '杳无音信'
匹配: True

目标: '因材施教'
预测: '因材施教'
匹配: True

目标: '1999年'
预测: '1999年'
匹配: True

目标: '香港新界元朗区天水围北天瑞路天恒邨多层停车场底层的公共运输交汇处'
预测: '2001年4月10日起开办。取代276／276p线于天水围的巴士服务'
匹配: False



,Accuracy/EM,F1,EM
Classify,0.187781,0.152339,NaN
NLI,0.554961,0.404914,NaN
MRC,NaN,0.849417,0.709447


...

在稳定期内(第2/2个epoch)，保持权重不变
当前任务权重: {'classify': 20.0, 'nli': 5.0, 'mrc': 0.0018988096743447413}
当前采样权重: {'classify': 8.0, 'nli': 3.0, 'mrc': 0.0009494048371723706}
Early stopping triggered after 2 epochs
加载最佳模型用于最终评估: /root/Code/Multi-task Project/test_outputs/multitask_model.pth


Evaluating MRC: 100%|██████████| 141/141 [00:12<00:00, 10.97it/s]


【MRC诊断信息】
样本数: 1122
精确匹配率: 0.7264
平均F1分数: 0.8950
部分匹配率: 0.9073

【预测样例】
目标: '摩托车速度非常快'
预测: '摩托车速度非常快'
匹配: True

目标: '杳无音信'
预测: '杳无音信'
匹配: True

目标: '因材施教'
预测: '因材施教'
匹配: True

目标: '1999年'
预测: '1999年'
匹配: True

目标: '香港新界元朗区天水围北天瑞路天恒邨多层停车场底层的公共运输交汇处'
预测: '位'
匹配: False



,Accuracy/EM,F1,EM
Classify,0.377307,0.348260,NaN
NLI,0.704280,0.692361,NaN
MRC,NaN,0.895006,0.726381


Task-specific label maps saved for inference
Model configuration saved


# 7 